[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PPatty666/Germany_Flood_Study/blob/main/Visuals/visual_scripts/Cleaning_up_Germany_Gemeinde_Climate_Scenarios.ipynb)

In [ ]:
%%capture

!pip install fiona
!pip install folium mapclassify
!pip install ipyleaflet
!pip install -U kaleido
!pip install contextily
!pip install git+https://github.com/pmdscully/geo_northarrow.git
!pip install matplotlib_scalebar

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib.ticker as ticker
from matplotlib.cm import ScalarMappable
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from matplotlib.legend import Legend
from matplotlib.colors import to_hex
from matplotlib.ticker import FuncFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from moviepy.editor import ImageSequenceClip
from PIL import Image
from IPython.display import Video

from ipyleaflet import Map, GeoJSON, Popup
from ipywidgets import HTML
import json
import folium
from folium import IFrame

import re
import math
from numpy.random import default_rng
import scipy
from scipy.stats import gamma
import statsmodels.api as sm
import arviz as az
import pymc as pm
import xarray as xr

import geopandas as gpd
import fiona
from shapely.geometry import MultiLineString, LineString
from shapely.geometry import Polygon, MultiPolygon
import contextily as ctx
from geo_northarrow import add_north_arrow
from matplotlib_scalebar.scalebar import ScaleBar
import mapclassify
from mapclassify import NaturalBreaks
from mapclassify import Quantiles
from matplotlib.lines import Line2D


from shapely.ops import unary_union

import pickle
import os

import warnings
warnings.filterwarnings('ignore')

## Find the flooding exposure scenario for the year 2020 (based on a 100-year return period) to use as the baseline for comparison

In [ ]:
!unzip "/content/drive/MyDrive/Germany_Flood_Study/Germany_flood_study_raw_csv/Gemeinde_all_return_periods.zip"

# SSP1-RCP2.6 (Most Optimistic)

### Read files, pre-processing and store

In [ ]:
!unzip "/content/drive/MyDrive/Germany_Flood_Study/Germany_flood_study_raw_csv/most_optimistic.zip"

In [ ]:
file_paths = [
    '/content/Germany_Gemeinde_2020_100.xlsx',
    '/content/most_optimistic/Germany_Gemeinde_2030_100_26.xlsx',
    '/content/most_optimistic/Germany_Gemeinde_2050_100_26.xlsx',
    '/content/most_optimistic/Germany_Gemeinde_2080_100_26.xlsx',
]

dataframes = []
for path in file_paths:
    df = pd.read_excel(path)
    df = df.round(6)
    dataframes.append(df)

# Optionally unpack into variables
(Gemeinde_2020_100_raw, Gemeinde_2030_100_26_raw, Gemeinde_2050_100_26_raw, Gemeinde_2080_100_26_raw) = dataframes

In [ ]:
Gemeinde_2020_100_raw.head()

In [ ]:
Gemeinde_2030_100_26_raw.head()

In [ ]:
# Date range: every 6 months
dates = pd.date_range(start='2016-07-01', end='2025-01-01', freq='6MS')

# Prefixes and suffixes
prefixes = ['CU', 'FU', 'PD', 'MAX']
suffixes = ['P0', 'lt0.15', 'lt0.5', 'lt1.5', 'gt1.5']

# Correct order: by prefix, then suffix, then date
all_columns = [
    f"{prefix}-{date.strftime('%b-%y')}-{suffix}"
    for prefix in prefixes
    for suffix in suffixes
    for date in dates
]

In [ ]:
# Check if number of replacement columns matches the shape
for df in [Gemeinde_2020_100_raw, Gemeinde_2030_100_26_raw, Gemeinde_2050_100_26_raw, Gemeinde_2080_100_26_raw]:
    if len(all_columns) == df.shape[1] - 3:
      df.columns = list(df.columns[:3]) + all_columns
      df.reset_index(drop=True, inplace=True)
      df['id'] = df.index
    else:
      raise ValueError("Length of generated column names does not match number of columns (excluding the first one).")


In [ ]:
Gemeinde_2030_100_26_raw.head()

In [ ]:
Gemeinde_2020_100_raw = Gemeinde_2020_100_raw[1:]
Gemeinde_2030_100_26_raw = Gemeinde_2030_100_26_raw[1:]
Gemeinde_2050_100_26_raw = Gemeinde_2050_100_26_raw[1:]
Gemeinde_2080_100_26_raw = Gemeinde_2080_100_26_raw[1:]

In [ ]:
# Pre-processed files should be stored in pickle format to enable direct reloading, particularly in cases where the session must be restarted due to memory limitations.
save_path = "/content/"

# Make sure the directory exists

os.makedirs(save_path, exist_ok=True)

gemeinde_dfs = [Gemeinde_2020_100_raw, Gemeinde_2030_100_26_raw, Gemeinde_2050_100_26_raw, Gemeinde_2080_100_26_raw]

filenames = [
    "Gemeinde_2020_100_raw.pkl", "Gemeinde_2030_100_26_raw.pkl", "Gemeinde_2050_100_26_raw.pkl", "Gemeinde_2080_100_26_raw.pkl"
]

for df, filename in zip(gemeinde_dfs, filenames):
    df.to_pickle(os.path.join(save_path, filename))


### **PLEASE re-run if the session restarts before you finish processing this climate scenario:** Read and load file from here

In [ ]:
# Define the path where the files are stored
save_path = "/content/"

# List of filenames
filenames = [
    "Gemeinde_2020_100_raw.pkl", "Gemeinde_2030_100_26_raw.pkl", "Gemeinde_2050_100_26_raw.pkl", "Gemeinde_2080_100_26_raw.pkl"
]

# Load all DataFrames into a list
gemeinde_dfs = [pd.read_pickle(os.path.join(save_path, filename)) for filename in filenames]

cols_to_convert = [col for col in gemeinde_dfs[0].columns if col not in ['Gemeinde', 'Kreis', 'Land', 'id']]

def convert_and_round(df):
    df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce').round(6)
    return df

gemeinde_dfs = [convert_and_round(df) for df in gemeinde_dfs]

# Unpack if needed
(
  Gemeinde_2020_100_raw,
  Gemeinde_2030_100_26_raw,
  Gemeinde_2050_100_26_raw,
  Gemeinde_2080_100_26_raw
) = gemeinde_dfs


In [ ]:
# Date range: every 6 months
dates = pd.date_range(start='2016-07-01', end='2025-01-01', freq='6MS')

# Prefixes and suffixes
prefixes = ['CU', 'FU', 'PD', 'MAX']
suffixes = ['P0', 'lt0.15', 'lt0.5', 'lt1.5', 'gt1.5']

# Correct order: by prefix, then suffix, then date
all_columns = [
    f"{prefix}-{date.strftime('%b-%y')}-{suffix}"
    for prefix in prefixes
    for suffix in suffixes
    for date in dates
]

## Combined flooding (max)

### Data preparation

In [ ]:
# List of years to process
years = [2020, 2030, 2050, 2080]

# Columns you want to select (CU columns and Land)
max_columns = ['id', 'Gemeinde', 'Kreis', 'Land'] + [col for col in all_columns if col.startswith('MAX') or col == 'Land']

# Loop through each year and modify the corresponding DataFrame
for year in years:
    if year == 2020:
        df_name = f'Gemeinde_{year}_100_raw'  # 2020 uses '_100_raw'
    else:
        df_name = f'Gemeinde_{year}_100_26_raw'  # 2030, 2050, and 2080 use '_100_85_raw'

    # Check if the DataFrame exists before proceeding
    if df_name in globals():
        # Get the raw DataFrame and select the relevant columns for max flooding data
        df_raw = globals()[df_name]
        globals()[f'Gemeinde_{year}_100_26_MAX'] = df_raw[max_columns]
    else:
        print(f'{df_name} does not exist.')


In [ ]:
len(Gemeinde_2020_100_26_MAX) == len(Gemeinde_2030_100_26_MAX)

In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    globals()[f'Gemeinde_{y}_100_26_MAX_long'] = globals()[f'Gemeinde_{y}_100_26_MAX'].melt(
        id_vars=['id', 'Gemeinde', 'Kreis', 'Land'], var_name='month_year_depth', value_name='MAX_area'
    )


In [ ]:
for df in [Gemeinde_2020_100_26_MAX_long, Gemeinde_2030_100_26_MAX_long, Gemeinde_2050_100_26_MAX_long, Gemeinde_2080_100_26_MAX_long]:

    # Replace 'CU' with 'MAX' if needed (optional step if not already 'MAX')
    # df['month_year_depth'] = df['month_year_depth'].str.replace('CU', 'MAX')

    # Split the 'month_year_depth' column into parts
    df[['prefix', 'month_year', 'depth_cat']] = df['month_year_depth'].str.extract(r'^(MAX)-([A-Za-z]{3}-\d{2})-(.+)$')

    # Drop 'prefix' if not needed
    df.drop(columns='prefix', inplace=True)


In [ ]:
# Define your DataFrames with their corresponding years
year_df_map = {
    2020: Gemeinde_2020_100_26_MAX_long,
    2030: Gemeinde_2030_100_26_MAX_long,
    2050: Gemeinde_2050_100_26_MAX_long,
    2080: Gemeinde_2080_100_26_MAX_long
}

# Loop through each DataFrame
for year, df in year_df_map.items():

    # Filter to only Jan-25
    jan25_df = df[df['month_year'] == 'Jan-25'].reset_index(drop=True)

    # Set the new global variable
    globals()[f'Gemeinde_{year}_100_26_MAX_Jan25'] = jan25_df


In [ ]:
# Second part: group and create *_settle DataFrames
for year in [2020, 2030, 2050, 2080]:
    jan25_var_name = f'Gemeinde_{year}_100_26_MAX_Jan25'

    if jan25_var_name in globals():
        jan25_df = globals()[jan25_var_name]

        # Group and sum by 'id' and 'month_year'
        settle_df = jan25_df.groupby(['id', 'month_year'], as_index=False)['MAX_area'].sum()
        settle_df.columns = ['id', 'month_year', 'settle_area']
        settle_df['settle_area'] = settle_df['settle_area'].astype(float)

        # Save to a new global variable
        globals()[f'{jan25_var_name}_settle'] = settle_df

In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    long_df = globals()[f'Gemeinde_{y}_100_26_MAX_Jan25']
    settle_df = globals()[f'Gemeinde_{y}_100_26_MAX_Jan25_settle']

    merged_df = pd.merge(long_df, settle_df, on=['id', 'month_year'], how='left')

    globals()[f'Gemeinde_{y}_100_26_MAX_Jan25'] = merged_df


In [ ]:
# Filter out rows where Gemeinde == 'Germany'
filtered = Gemeinde_2020_100_26_MAX_Jan25[Gemeinde_2020_100_26_MAX_Jan25.Gemeinde != 'Germany']
filtered.settle_area = filtered.settle_area.round(3)

# Group and count
grouped = filtered.groupby(['Gemeinde', 'Kreis', 'Land', 'settle_area']).size().reset_index(name='count')

# Check for duplicates (count > 1)
duplicates = grouped[grouped['count'] > 5]

print(f"Number of duplicate combinations: {len(duplicates)}")
print(duplicates)


In [ ]:
duplicate_rows = filtered[
    (filtered['Gemeinde'] == 'Helgoland') &
    (filtered['Kreis'] == 'Pinneberg') &
    (filtered['Land'] == 'Schleswig-Holstein')
]

print(duplicate_rows)

In [ ]:
# Add suffix based on id
Gemeinde_2020_100_26_MAX_Jan25.loc[
    Gemeinde_2020_100_26_MAX_Jan25['id'] == 9799, 'Gemeinde'
] = Gemeinde_2020_100_26_MAX_Jan25.loc[
    Gemeinde_2020_100_26_MAX_Jan25['id'] == 9799, 'Gemeinde'
] + '-1'

Gemeinde_2020_100_26_MAX_Jan25.loc[
    Gemeinde_2020_100_26_MAX_Jan25['id'] == 9800, 'Gemeinde'
] = Gemeinde_2020_100_26_MAX_Jan25.loc[
    Gemeinde_2020_100_26_MAX_Jan25['id'] == 9800, 'Gemeinde'
] + '-2'


In [ ]:
Gemeinde_2020_100_26_MAX_Jan25[
    (Gemeinde_2020_100_26_MAX_Jan25['Gemeinde'] == 'Helgoland-2') &
    (Gemeinde_2020_100_26_MAX_Jan25['Kreis'] == 'Pinneberg') &
    (Gemeinde_2020_100_26_MAX_Jan25['Land'] == 'Schleswig-Holstein')
]

### Check admin boundary matching

In [ ]:
%%capture
!unzip /content/drive/MyDrive/Germany_Flood_Study/Germany_flood_study_raw_csv/vg250_01-01.gk3.shape.ebenen.zip -d admin-GIS

In [ ]:
admin3_boundary = gpd.read_file('/content/admin-GIS/vg250_01-01.gk3.shape.ebenen/vg250_ebenen_0101/VG250_GEM.shp')

In [ ]:
!unzip "/content/drive/MyDrive/Germany_Flood_Study/Germany_flood_study_raw_csv/Gemeinde_Stats_all_return_periods.zip"

In [ ]:
Gemeinde_2020_100_raw_exposure = pd.read_csv('/content/GermanyStats_2020_100.csv', encoding='ISO-8859-1').round(6)

In [ ]:
Gemeinde_2020_100_raw_exposure.head()

In [ ]:
# Create the list of new month_year labels (6-month intervals)
month_years = pd.date_range('2016-07-01', '2025-01-01', freq='6MS').strftime('%b-%y').tolist()
print(month_years)

In [ ]:
# Get current column names
cols = Gemeinde_2020_100_raw_exposure.columns.tolist()

In [ ]:
# Update column names only if they end with _01 to _018
new_cols = []
for col in cols:
    match = re.search(r'_(\d{2})$', col)
    if match:
        suffix_num = int(match.group(1))
        if 1 <= suffix_num <= len(month_years):
            new_suffix = month_years[suffix_num - 1]  # map _01 to Jul-16, etc.
            col = re.sub(r'_(\d{2})$', f'_{new_suffix}', col)
    new_cols.append(col)

In [ ]:
# Assign new column names back to the DataFrame
Gemeinde_2020_100_raw_exposure.columns = new_cols

### Settlement area

In [ ]:
Gemeinde_2020_100_raw_exposure.head()

In [ ]:
# Select columns that start with 'CU' or are exactly 'Land'
max_columns = new_cols[:7] + [col for col in new_cols if col.startswith('SA_maxRisk')]


Gemeinde_2020_100_MAX_exposure = Gemeinde_2020_100_raw_exposure[max_columns]

In [ ]:
Gemeinde_2020_100_MAX_exposure_long = Gemeinde_2020_100_MAX_exposure.melt(
        id_vars=new_cols[:7], var_name='month_year_depth', value_name='MAX_area'
    )

In [ ]:
# Extract depth category (e.g., '0', '015', '050', '150', '150p') and month-year (e.g., 'Jul-16')
Gemeinde_2020_100_MAX_exposure_long[['depth_cat', 'month_year']] = (
    Gemeinde_2020_100_MAX_exposure_long['month_year_depth']
    .str.extract(r'SA_maxRisk_([^_]+)_([A-Za-z]{3}-\d{2})')
)

In [ ]:
Gemeinde_2020_100_MAX_exposure_long.head()

In [ ]:
Gemeinde_2020_100_MAX_exposure_jan25 = Gemeinde_2020_100_MAX_exposure_long[Gemeinde_2020_100_MAX_exposure_long['month_year'] == 'Jan-25'].reset_index(drop=True)

In [ ]:
Gemeinde_2020_100_MAX_exposure_jan25.head()

In [ ]:
GEM_ID_settle_df = Gemeinde_2020_100_MAX_exposure_jan25.groupby(['GEM_ID', 'month_year'], as_index=False)['MAX_area'].sum()

In [ ]:
GEM_ID_settle_df.columns = ['GEM_ID', 'month_year', 'settle_area']

In [ ]:
# Select columns that start with 'CU' or are exactly 'Land'
sa_columns = new_cols[:25]

Gemeinde_2020_100_SA = Gemeinde_2020_100_raw_exposure[sa_columns]

In [ ]:
Gemeinde_2020_100_SA.head()

In [ ]:
Gemeinde_2020_100_SA_jan25 = Gemeinde_2020_100_SA[["GEM_ID", "GEM_NAME",	"KRS_ID",	"KRS_NAME",	"LAN_ID",	"LAN_NAME", "SA_Jan-25"]]

In [ ]:
Gemeinde_2020_100_SA_jan25.head()

In [ ]:
GEM_ID_settle_df = GEM_ID_settle_df.merge(Gemeinde_2020_100_SA_jan25, on='GEM_ID', how='left')

In [ ]:
# Filter out rows where Gemeinde == 'Germany'
filtered = GEM_ID_settle_df[GEM_ID_settle_df.GEM_NAME != 'Germany']
filtered["settle_area"] = filtered["settle_area"].round(6)

# Group and count
grouped = filtered.groupby(['GEM_NAME', 'KRS_NAME', 'LAN_NAME', 'settle_area']).size().reset_index(name='count')

# Check for duplicates (count > 1)
duplicates = grouped[grouped['count'] > 1]

print(f"Number of duplicate combinations: {len(duplicates)}")
print(duplicates)


In [ ]:
GEM_ID_settle_df[
    (GEM_ID_settle_df['GEM_NAME'] == 'Helgoland') &
    (GEM_ID_settle_df['KRS_NAME'] == 'Pinneberg') &
    (GEM_ID_settle_df['LAN_NAME'] == 'Schleswig-Holstein')
]

In [ ]:
# Add suffix based on id
GEM_ID_settle_df.loc[
    GEM_ID_settle_df['GEM_ID'] == 'DEBKGVG2000000C3', 'GEM_NAME'
] = GEM_ID_settle_df.loc[
    GEM_ID_settle_df['GEM_ID'] == 'DEBKGVG2000000C3', 'GEM_NAME'
] + '-1'

GEM_ID_settle_df.loc[
    GEM_ID_settle_df['GEM_ID'] == 'DEBKGVG2000008HV', 'GEM_NAME'
] = GEM_ID_settle_df.loc[
    GEM_ID_settle_df['GEM_ID'] == 'DEBKGVG2000008HV', 'GEM_NAME'
] + '-2'


In [ ]:
GEM_ID_settle_df[
    (GEM_ID_settle_df['GEM_NAME'] == 'Helgoland') &
    (GEM_ID_settle_df['KRS_NAME'] == 'Pinneberg') &
    (GEM_ID_settle_df['LAN_NAME'] == 'Schleswig-Holstein')
]

In [ ]:
GEM_ID_settle_df.GEM_ID.nunique()

In [ ]:
Gemeinde_2020_100_26_MAX_Jan25.id.nunique()

In [ ]:
Gemeinde_2020_100_26_MAX_Jan25.columns

In [ ]:
Gemeinde_2020_100_26_MAX_Jan25["settle_area_round"] = Gemeinde_2020_100_26_MAX_Jan25.settle_area.round(6)
GEM_ID_settle_df["settle_area_round"] = GEM_ID_settle_df.settle_area.round(6)
merged_df = pd.merge(
    Gemeinde_2020_100_26_MAX_Jan25,
    GEM_ID_settle_df,
    left_on=['Gemeinde', 'Kreis', 'Land', 'settle_area_round'],
    right_on=['GEM_NAME', 'KRS_NAME', 'LAN_NAME', 'settle_area_round'],
    how='left'
)

In [ ]:
merged_df.GEM_ID.nunique()

### Continue climate scenario analysis

In [ ]:
Gemeinde_2020_100_26_MAX_Jan25 = merged_df.copy()

In [ ]:
years = [2030, 2050, 2080]

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_26_MAX_Jan25']
    base_df = Gemeinde_2020_100_26_MAX_Jan25
    base_df[f'diff_{year}'] = current_df.MAX_area - base_df.MAX_area

In [ ]:
years = [2030, 2050, 2080]

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_26_MAX_Jan25']
    base_df = Gemeinde_2020_100_26_MAX_Jan25
    base_df[f'pct_diff_{year}'] = (current_df.MAX_area - base_df.MAX_area)/(base_df.MAX_area + 1e-6)

In [ ]:
summary = Gemeinde_2020_100_26_MAX_Jan25[['id', 'GEM_ID', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates().copy()
years = [2030, 2050, 2080]

for year in years:
    diff_col = f'diff_{year}'

    pivot = (
        Gemeinde_2020_100_26_MAX_Jan25
        .groupby(['id', 'depth_cat'])[diff_col]
        .sum()
        .unstack(fill_value=0)
        .add_prefix(f'sum_diff_{year}_')  # e.g., sum_diff_2030_lt0.5
        .reset_index()
    )

    summary = summary.merge(pivot, on='id', how='left')


In [ ]:
# Convert all newly added columns to float
new_cols = [col for col in summary.columns if col.startswith('sum_diff_')]
summary[new_cols] = summary[new_cols].astype(float)

In [ ]:
summary_26 = summary.copy()

In [ ]:
# Step 1: Calculate total MAX_area per id and depth_cat
area_pivot = (
    Gemeinde_2020_100_26_MAX_Jan25
    .groupby(['id', 'depth_cat'])['MAX_area']
    .sum()
    .unstack(fill_value=0)
    .add_prefix('area_')
    .reset_index()
)

# Step 2: Merge into summary_26
summary_26 = summary_26.merge(area_pivot, on='id', how='left')

# Step 3: Calculate percentage risk levels using area for each category
years = [2030, 2050, 2080]
for year in years:
    # Absolute difference sums (already calculated earlier)
    summary_26[f'high_risk_{year}'] = (
        summary_26.get(f'sum_diff_{year}_lt0.5', 0) +
        summary_26.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_26.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_26[f'very_high_risk_{year}'] = (
        summary_26.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_26.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_26[f'extreme_risk_{year}'] = summary_26.get(f'sum_diff_{year}_gt1.5', 0)

    # Denominators from MAX_area sums
    total_area = (
        summary_26.get('area_lt0.5', 0) +
        summary_26.get('area_lt1.5', 0) +
        summary_26.get('area_gt1.5', 0)
    ).replace(0, 1e-6)

    very_high_area = (
        summary_26.get('area_lt1.5', 0) +
        summary_26.get('area_gt1.5', 0)
    ).replace(0, 1e-6)

    extreme_area = summary_26.get('area_gt1.5', 0).replace(0, 1e-6)

    # Percentage risk levels
    summary_26[f'pct_high_risk_{year}'] = (
        summary_26[f'high_risk_{year}'] / total_area * 100
    ).round(2)

    summary_26[f'pct_very_high_risk_{year}'] = (
        summary_26[f'very_high_risk_{year}'] / very_high_area * 100
    ).round(2)

    summary_26[f'pct_extreme_risk_{year}'] = (
        summary_26[f'extreme_risk_{year}'] / extreme_area * 100
    ).round(2)


In [ ]:
summary_26.columns

In [ ]:
value_vars = [
    'high_risk_2030', 'very_high_risk_2030', 'extreme_risk_2030',
    'high_risk_2050', 'very_high_risk_2050', 'extreme_risk_2050',
    'high_risk_2080', 'very_high_risk_2080', 'extreme_risk_2080'
]

# 1. Melt the absolute values
summary_diff_long = summary_26.melt(
    id_vars=['id', 'GEM_ID', 'Gemeinde', 'Kreis', 'Land'],
    value_vars=value_vars,
    var_name='risk_year',
    value_name='max_diff_2020'
)

# 2. Melt the percent difference columns
summary_pct_long = summary_26.melt(
    id_vars=['id', 'GEM_ID', 'Gemeinde', 'Kreis', 'Land'],
    value_vars=[f'pct_{v}' for v in value_vars],
    var_name='risk_year',
    value_name='max_pct_diff_2020'
)

# 3. Clean `risk_year` for both (they must match for merge)
summary_pct_long['risk_year'] = summary_pct_long['risk_year'].str.replace('pct_', '')

# 4. Merge the two melted DataFrames
summary_26_long = pd.merge(
    summary_diff_long,
    summary_pct_long,
    on=['id', 'GEM_ID', 'Gemeinde', 'Kreis', 'Land', 'risk_year']
)

# 5. Extract risk level and year
summary_26_long[['risk_level', 'year']] = summary_26_long['risk_year'].str.extract(r'(.*)_(\d{4})')
summary_26_long['year'] = summary_26_long['year'].astype(int)


### Please store the pre-processed files as pickle files, since you’ll need to load them directly later—especially if the session needs to be restarted due to running out of RAM.

In [ ]:
summary_26_long.to_pickle("max_summary_26_long.pkl")

In [ ]:
with open("max_summary_26_long.pkl", "rb") as f:
    summary_26_long = pickle.load(f)

### Flooding exposure difference map (future years vs. 2020) - prep

In [ ]:
summary_26_long.risk_level.unique()

In [ ]:
for year in [2030, 2050, 2080]:
    globals()[f"summary_26_above_015_{year}"] = summary_26_long[
        (summary_26_long['risk_level'] == 'high_risk') &
        (summary_26_long['Gemeinde'] != 'Germany') &
        (summary_26_long['year'] == year)
    ].copy()


In [ ]:
summary_26_above_015_2030.GEM_ID.nunique()

In [ ]:
len(summary_26_above_015_2030)

In [ ]:
years = [2030, 2050, 2080]

for y in years:
    summary_26_above_015_df = globals()[f'summary_26_above_015_{y}']

    merged_gdf = admin3_boundary.merge(summary_26_above_015_df, left_on='OBJID', right_on='GEM_ID', how='right')

    globals()[f'summary_26_above_015_{y}_gdf'] = merged_gdf

In [ ]:
years = [2030, 2050, 2080]

for y in years:
    summary_26_above_015_gdf = globals()[f'summary_26_above_015_{y}_gdf']
    print(f"Number of null geometries in summary_26_above_015_{y}_gdf: {len(summary_26_above_015_gdf[summary_26_above_015_gdf.geometry.isna()])}")
    print(f"Number of unique geometries in summary_26_above_015_{y}_gdf: {len(summary_26_above_015_gdf.geometry.unique())}")

In [ ]:
summary_26_above_015_2030_gdf.columns

In [ ]:
# Summary statistics for table in appendix
years = [2030, 2050, 2080]
results = {}

for year in years:
    df_name = f"summary_26_above_015_{year}_gdf"
    df = globals()[df_name]   # fetch DataFrame by name

    positive_sum = df.loc[df.max_diff_2020 > 0].max_diff_2020.sum().round(2)
    negative_sum = df.loc[df.max_diff_2020 < 0].max_diff_2020.sum().round(2)

    results[year] = {"positive_sum": positive_sum, "negative_sum": negative_sum}

results


### Percentage difference map

In [ ]:
# Years to plot
years = [2030, 2050, 2080]
save_path = "/content/drive/MyDrive/Germany_Flood_Study/Gemeinde_climate_scenario_maps/final_adjustments"
os.makedirs(save_path, exist_ok=True)

for y in years:
    gdf = globals()[f'summary_26_above_015_{y}_gdf'].copy()
    gdf = gdf.to_crs(epsg=3857)

    # Mask 0 values for transparency
    gdf['masked_pct_diff'] = gdf['max_pct_diff_2020'].replace(0, np.nan)

    # Colors & breaks
    custom_colors = [
        '#40e0d0',  # < -10%
        '#00ffff',  # -10% ~ -1%
        '#808080',  # -1% ~ 1%
        '#fee5d9',  # 1% ~ 10%
        '#fb6a4a',  # 10% ~ 50%
        '#de2d26',  # 50% ~ 100%
        '#a50f15'   # > 100%
    ]
    custom_breaks = [-np.inf, -10, -1, 1, 10, 50, 100, np.inf]

    cmap = ListedColormap(custom_colors)
    norm = BoundaryNorm(custom_breaks, ncolors=len(custom_colors), clip=False)

    # Create figure and axis INSIDE loop
    fig, ax = plt.subplots(figsize=(14, 12))

    # Plot
    gdf.plot(
        ax=ax,
        column='masked_pct_diff',
        cmap=cmap,
        norm=norm,
        edgecolor=None,
        linewidth=0,
        legend=False
    )

    # Overlay geometry boundaries
    gdf.boundary.plot(ax=ax, color='grey', linewidth=0.05)

    # Basemap
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.PositronNoLabels, crs=gdf.crs)

    # Cleanup
    ax.axis('off')

    # North arrow & scale bar
    add_north_arrow(ax, scale=0.75, xlim_pos=0.9025, ylim_pos=0.9,
                    color='black', text_scaler=4, text_yT=-1.25)
    scale1 = ScaleBar(dx=1, location="lower right", scale_loc="bottom")
    ax.add_artist(scale1)

    # Custom legend
    legend_labels = [
        "<-10%", "-10% ~ -1%",
        "-1% ~ 1%",
        "1% ~ 10%", "10% ~ 50%", "50% ~ 100%", ">100%"
    ]
    circles = [
        Line2D([0], [0], marker='o', color='None',
              markerfacecolor=custom_colors[i],
              markeredgecolor='None',
              markersize=10,
              label=legend_labels[i])
        for i in range(len(legend_labels))
    ]
    ax.legend(
        handles=circles,
        title="Exposure change",
        loc='upper left',
        fontsize=12,
        title_fontsize=14
    )

    # Save
    file_name = f"gemeinde_exposure_pct_change_high_risk_26_{y}yr_separate_dry_wet_cyansv2.png"
    full_path = os.path.join(save_path, file_name)
    plt.tight_layout()
    # plt.savefig(full_path, dpi=300, bbox_inches='tight')
    plt.show()
    # plt.close()


In [ ]:
#scripts for the summary table in the appendix
years = [2030, 2050, 2080]
summary_list = []

for y in years:
    gdf = globals()[f'summary_26_above_015_{y}_gdf']

    # Count how many Gemeinden are below and above zero
    below_zero = (gdf['max_diff_2020'] < 0).sum()
    above_zero = (gdf['max_diff_2020'] > 0).sum()

    # Distribution stats for each group
    below_zero_desc = gdf.loc[gdf['max_diff_2020'] < 0, 'max_diff_2020'].describe()
    above_zero_desc = gdf.loc[gdf['max_diff_2020'] > 0, 'max_diff_2020'].describe()

    summary_list.append({
        'Year': y,
        'Below Zero Count': below_zero,
        'Above Zero Count': above_zero,
        'Below Zero Min': below_zero_desc['min'],
        'Below Zero Median': below_zero_desc['50%'],
        'Below Zero Max': below_zero_desc['max'],
        'Above Zero Min': above_zero_desc['min'],
        'Above Zero Median': above_zero_desc['50%'],
        'Above Zero Max': above_zero_desc['max']
    })

# Create a DataFrame with all years
summary_df = pd.DataFrame(summary_list)
summary_df


## Coastal flooding undefended

### Data Preparation

In [ ]:
# List of years to process
years = [2020, 2030, 2050, 2080]

# Columns you want to select
cu_columns = ['id', 'Gemeinde', 'Kreis', 'Land'] + [col for col in all_columns if col.startswith('CU')]

# Loop through each year and modify the corresponding DataFrame
for year in years:
    if year == 2020:
        df_name = f'Gemeinde_{year}_100_raw'  # 2020 uses '_100_raw'
    else:
        df_name = f'Gemeinde_{year}_100_26_raw'  # 2030, 2050, and 2080 use '_100_85_raw'

    # Check if the DataFrame exists before proceeding
    if df_name in globals():
        # Get the raw DataFrame and select the relevant columns for max flooding data
        df_raw = globals()[df_name]
        globals()[f'Gemeinde_{year}_100_26_CU'] = df_raw[cu_columns]
    else:
        print(f'{df_name} does not exist.')


In [ ]:
len(Gemeinde_2020_100_26_CU) == len(Gemeinde_2030_100_26_CU)

In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    globals()[f'Gemeinde_{y}_100_26_CU_long'] = globals()[f'Gemeinde_{y}_100_26_CU'].melt(
        id_vars=['id', 'Gemeinde', 'Kreis', 'Land'], var_name='month_year_depth', value_name='CU_area'
    )


In [ ]:
Gemeinde_2020_100_26_CU_long.head()

In [ ]:
for df in [Gemeinde_2020_100_26_CU_long, Gemeinde_2030_100_26_CU_long, Gemeinde_2050_100_26_CU_long, Gemeinde_2080_100_26_CU_long]:

    # Replace 'CU' with 'MAX' if needed (optional step if not already 'MAX')
    # df['month_year_depth'] = df['month_year_depth'].str.replace('CU', 'MAX')

    # Split the 'month_year_depth' column into parts
    df[['prefix', 'month_year', 'depth_cat']] = df['month_year_depth'].str.extract(r'^(CU)-([A-Za-z]{3}-\d{2})-(.+)$')

    # Drop 'prefix' if not needed
    df.drop(columns='prefix', inplace=True)


In [ ]:
Gemeinde_2080_100_26_CU_long.head()

In [ ]:
# Define your DataFrames with their corresponding years
year_df_map = {
    2020: Gemeinde_2020_100_26_CU_long,
    2030: Gemeinde_2030_100_26_CU_long,
    2050: Gemeinde_2050_100_26_CU_long,
    2080: Gemeinde_2080_100_26_CU_long
}

# Loop through each DataFrame
for year, df in year_df_map.items():

    # Filter to only Jan-25
    jan25_df = df[df['month_year'] == 'Jan-25'].reset_index(drop=True)

    # Set the new global variable
    globals()[f'Gemeinde_{year}_100_26_CU_Jan25'] = jan25_df


In [ ]:
# Second part: group and create *_settle DataFrames
for year in [2020, 2030, 2050, 2080]:
    jan25_var_name = f'Gemeinde_{year}_100_26_CU_Jan25'

    if jan25_var_name in globals():
        jan25_df = globals()[jan25_var_name]

        # Group and sum by 'id' and 'month_year'
        settle_df = jan25_df.groupby(['id', 'month_year'], as_index=False)['CU_area'].sum()
        settle_df.columns = ['id', 'month_year', 'settle_area']
        settle_df['settle_area'] = settle_df['settle_area'].astype(float)

        # Save to a new global variable
        globals()[f'{jan25_var_name}_settle'] = settle_df

In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    long_df = globals()[f'Gemeinde_{y}_100_26_CU_Jan25']
    settle_df = globals()[f'Gemeinde_{y}_100_26_CU_Jan25_settle']

    merged_df = pd.merge(long_df, settle_df, on=['id', 'month_year'], how='left')

    globals()[f'Gemeinde_{y}_100_26_CU_Jan25'] = merged_df


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    df = globals()[f'Gemeinde_{y}_100_26_CU_Jan25']
    globals()[f'Gemeinde_{y}_100_26_CU_Jan25'] = df
    df['settle_area_non_zero'] = df.settle_area.replace(0, 1e-6)
    globals()[f'Gemeinde_{y}_100_26_CU_Jan25']['settle_area_non_zero'] = df.settle_area_non_zero
    globals()[f'Gemeinde_{y}_100_26_CU_Jan25']['depth_cat_area_pct'] = (df.CU_area / df.settle_area_non_zero) * 100

In [ ]:
cols = ['id', 'Gemeinde', 'Kreis', 'Land']
years = [2030, 2050, 2080]

base_df = Gemeinde_2020_100_26_CU_Jan25

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_26_CU_Jan25']

    # Direct comparison of the column values (row by row, in order)
    is_same = base_df[cols].equals(current_df[cols])

    if is_same:
        print(f"✅ Columns match exactly for year {year}")
    else:
        print(f"❌ Columns differ for year {year}")


In [ ]:
years = [2030, 2050, 2080]

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_26_CU_Jan25']
    base_df = Gemeinde_2020_100_26_CU_Jan25
    base_df[f'diff_{year}'] = current_df.CU_area - base_df.CU_area

In [ ]:
summary = Gemeinde_2020_100_26_CU_Jan25[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates().copy()
years = [2030, 2050, 2080]

for year in years:
    diff_col = f'diff_{year}'

    pivot = (
        Gemeinde_2020_100_26_CU_Jan25
        .groupby(['id', 'depth_cat'])[diff_col]
        .sum()
        .unstack(fill_value=0)
        .add_prefix(f'sum_diff_{year}_')  # e.g., sum_diff_2030_lt0.5
        .reset_index()
    )

    summary = summary.merge(pivot, on='id', how='left')


In [ ]:
# Convert all newly added columns to float
new_cols = [col for col in summary.columns if col.startswith('sum_diff_')]
summary[new_cols] = summary[new_cols].astype(float)

In [ ]:
Gemeinde_2020_100_26_CU_Jan25[Gemeinde_2020_100_26_CU_Jan25.Gemeinde == 'Allmendingen']

In [ ]:
summary_26 = summary.copy()

In [ ]:
years = [2030, 2050, 2080]

for year in years:
    summary_26[f'high_risk_{year}'] = (
        summary_26.get(f'sum_diff_{year}_lt0.5', 0) +
        summary_26.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_26.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_26[f'very_high_risk_{year}'] = (
        summary_26.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_26.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_26[f'extreme_risk_{year}'] = summary_26.get(f'sum_diff_{year}_gt1.5', 0)

In [ ]:
summary_26.columns

In [ ]:
value_vars = [
    'high_risk_2030', 'very_high_risk_2030', 'extreme_risk_2030',
    'high_risk_2050', 'very_high_risk_2050', 'extreme_risk_2050',
    'high_risk_2080', 'very_high_risk_2080', 'extreme_risk_2080'
]

cu_summary_26_long = summary_26.melt(
    id_vars=['id', 'Gemeinde', 'Kreis', 'Land'],  # keep these
    value_vars=value_vars,
    var_name='risk_year',
    value_name='cu_diff_2020'
)

# Split the combined column into risk_level and year
cu_summary_26_long[['risk_level', 'year']] = cu_summary_26_long['risk_year'].str.extract(r'(.*)_(\d{4})')
cu_summary_26_long['year'] = cu_summary_26_long['year'].astype(int)


In [ ]:
cu_summary_26_long

### Please store the pre-processed files as pickle files, since you’ll need to load them directly later—especially if the session needs to be restarted due to running out of RAM.

In [ ]:
cu_summary_26_long.to_pickle("cu_summary_26_long.pkl")

In [ ]:
with open("cu_summary_26_long.pkl", "rb") as f:
    cu_summary_26_long = pickle.load(f)

## Fluvial flooding undefended

### Data Preparation

In [ ]:
# List of years to process
years = [2020, 2030, 2050, 2080]

# Columns you want to select
fu_columns = ['id', 'Gemeinde', 'Kreis', 'Land'] + [col for col in all_columns if col.startswith('FU')]

# Loop through each year and modify the corresponding DataFrame
for year in years:
    if year == 2020:
        df_name = f'Gemeinde_{year}_100_raw'  # 2020 uses '_100_raw'
    else:
        df_name = f'Gemeinde_{year}_100_26_raw'  # 2030, 2050, and 2080 use '_100_85_raw'

    # Check if the DataFrame exists before proceeding
    if df_name in globals():
        # Get the raw DataFrame and select the relevant columns for max flooding data
        df_raw = globals()[df_name]
        globals()[f'Gemeinde_{year}_100_26_FU'] = df_raw[fu_columns]
    else:
        print(f'{df_name} does not exist.')


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    globals()[f'Gemeinde_{y}_100_26_FU_long'] = globals()[f'Gemeinde_{y}_100_26_FU'].melt(
        id_vars=['id', 'Gemeinde', 'Kreis', 'Land'], var_name='month_year_depth', value_name='FU_area'
    )


In [ ]:
Gemeinde_2020_100_26_FU_long.head()

In [ ]:
for df in [Gemeinde_2020_100_26_FU_long, Gemeinde_2030_100_26_FU_long, Gemeinde_2050_100_26_FU_long, Gemeinde_2080_100_26_FU_long]:

    # Replace 'CU' with 'MAX' if needed (optional step if not already 'MAX')
    # df['month_year_depth'] = df['month_year_depth'].str.replace('CU', 'MAX')

    # Split the 'month_year_depth' column into parts
    df[['prefix', 'month_year', 'depth_cat']] = df['month_year_depth'].str.extract(r'^(FU)-([A-Za-z]{3}-\d{2})-(.+)$')

    # Drop 'prefix' if not needed
    df.drop(columns='prefix', inplace=True)


In [ ]:
Gemeinde_2080_100_26_FU_long.head()

In [ ]:
# Define your DataFrames with their corresponding years
year_df_map = {
    2020: Gemeinde_2020_100_26_FU_long,
    2030: Gemeinde_2030_100_26_FU_long,
    2050: Gemeinde_2050_100_26_FU_long,
    2080: Gemeinde_2080_100_26_FU_long
}

# Loop through each DataFrame
for year, df in year_df_map.items():

    # Filter to only Jan-25
    jan25_df = df[df['month_year'] == 'Jan-25'].reset_index(drop=True)

    # Set the new global variable
    globals()[f'Gemeinde_{year}_100_26_FU_Jan25'] = jan25_df


In [ ]:
# Second part: group and create *_settle DataFrames
for year in [2020, 2030, 2050, 2080]:
    jan25_var_name = f'Gemeinde_{year}_100_26_FU_Jan25'

    if jan25_var_name in globals():
        jan25_df = globals()[jan25_var_name]

        # Group and sum by 'id' and 'month_year'
        settle_df = jan25_df.groupby(['id', 'month_year'], as_index=False)['FU_area'].sum()
        settle_df.columns = ['id', 'month_year', 'settle_area']
        settle_df['settle_area'] = settle_df['settle_area'].astype(float)

        # Save to a new global variable
        globals()[f'{jan25_var_name}_settle'] = settle_df

In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    long_df = globals()[f'Gemeinde_{y}_100_26_FU_Jan25']
    settle_df = globals()[f'Gemeinde_{y}_100_26_FU_Jan25_settle']

    merged_df = pd.merge(long_df, settle_df, on=['id', 'month_year'], how='left')

    globals()[f'Gemeinde_{y}_100_26_FU_Jan25'] = merged_df


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    df = globals()[f'Gemeinde_{y}_100_26_FU_Jan25']
    globals()[f'Gemeinde_{y}_100_26_FU_Jan25'] = df
    df['settle_area_non_zero'] = df.settle_area.replace(0, 1e-6)
    globals()[f'Gemeinde_{y}_100_26_FU_Jan25']['settle_area_non_zero'] = df.settle_area_non_zero
    globals()[f'Gemeinde_{y}_100_26_FU_Jan25']['depth_cat_area_pct'] = (df.FU_area / df.settle_area_non_zero) * 100

In [ ]:
cols = ['id', 'Gemeinde', 'Kreis', 'Land']
years = [2030, 2050, 2080]

base_df = Gemeinde_2020_100_26_FU_Jan25

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_26_FU_Jan25']

    # Direct comparison of the column values (row by row, in order)
    is_same = base_df[cols].equals(current_df[cols])

    if is_same:
        print(f"✅ Columns match exactly for year {year}")
    else:
        print(f"❌ Columns differ for year {year}")


In [ ]:
years = [2030, 2050, 2080]

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_26_FU_Jan25']
    base_df = Gemeinde_2020_100_26_FU_Jan25
    base_df[f'diff_{year}'] = current_df.FU_area - base_df.FU_area

In [ ]:
summary = Gemeinde_2020_100_26_FU_Jan25[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates().copy()
years = [2030, 2050, 2080]

for year in years:
    diff_col = f'diff_{year}'

    pivot = (
        Gemeinde_2020_100_26_FU_Jan25
        .groupby(['id', 'depth_cat'])[diff_col]
        .sum()
        .unstack(fill_value=0)
        .add_prefix(f'sum_diff_{year}_')  # e.g., sum_diff_2030_lt0.5
        .reset_index()
    )

    summary = summary.merge(pivot, on='id', how='left')


In [ ]:
# Convert all newly added columns to float
new_cols = [col for col in summary.columns if col.startswith('sum_diff_')]
summary[new_cols] = summary[new_cols].astype(float)

In [ ]:
summary_26 = summary.copy()

In [ ]:
years = [2030, 2050, 2080]

for year in years:
    summary_26[f'high_risk_{year}'] = (
        summary_26.get(f'sum_diff_{year}_lt0.5', 0) +
        summary_26.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_26.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_26[f'very_high_risk_{year}'] = (
        summary_26.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_26.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_26[f'extreme_risk_{year}'] = summary_26.get(f'sum_diff_{year}_gt1.5', 0)

In [ ]:
summary_26.columns

In [ ]:
value_vars = [
    'high_risk_2030', 'very_high_risk_2030', 'extreme_risk_2030',
    'high_risk_2050', 'very_high_risk_2050', 'extreme_risk_2050',
    'high_risk_2080', 'very_high_risk_2080', 'extreme_risk_2080'
]

fu_summary_26_long = summary_26.melt(
    id_vars=['id', 'Gemeinde', 'Kreis', 'Land'],  # keep these
    value_vars=value_vars,
    var_name='risk_year',
    value_name='fu_diff_2020'
)

# Split the combined column into risk_level and year
fu_summary_26_long[['risk_level', 'year']] = fu_summary_26_long['risk_year'].str.extract(r'(.*)_(\d{4})')
fu_summary_26_long['year'] = fu_summary_26_long['year'].astype(int)


In [ ]:
fu_summary_26_long

### Please store the pre-processed files as pickle files, since you’ll need to load them directly later—especially if the session needs to be restarted due to running out of RAM.

In [ ]:
fu_summary_26_long.to_pickle("fu_summary_26_long.pkl")

In [ ]:
with open("fu_summary_26_long.pkl", "rb") as f:
    fu_summary_26_long = pickle.load(f)

## Pluvial flooding defended

### Data Preparation

In [ ]:
# List of years to process
years = [2020, 2030, 2050, 2080]

# Columns you want to select
pd_columns = ['id', 'Gemeinde', 'Kreis', 'Land'] + [col for col in all_columns if col.startswith('PD')]

# Loop through each year and modify the corresponding DataFrame
for year in years:
    if year == 2020:
        df_name = f'Gemeinde_{year}_100_raw'  # 2020 uses '_100_raw'
    else:
        df_name = f'Gemeinde_{year}_100_26_raw'  # 2030, 2050, and 2080 use '_100_85_raw'

    # Check if the DataFrame exists before proceeding
    if df_name in globals():
        # Get the raw DataFrame and select the relevant columns for max flooding data
        df_raw = globals()[df_name]
        globals()[f'Gemeinde_{year}_100_26_PD'] = df_raw[pd_columns]
    else:
        print(f'{df_name} does not exist.')


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    globals()[f'Gemeinde_{y}_100_26_PD_long'] = globals()[f'Gemeinde_{y}_100_26_PD'].melt(
        id_vars=['id', 'Gemeinde', 'Kreis', 'Land'], var_name='month_year_depth', value_name='PD_area'
    )


In [ ]:
Gemeinde_2020_100_26_PD_long.head()

In [ ]:
for df in [Gemeinde_2020_100_26_PD_long, Gemeinde_2030_100_26_PD_long, Gemeinde_2050_100_26_PD_long, Gemeinde_2080_100_26_PD_long]:

    # Replace 'CU' with 'MAX' if needed (optional step if not already 'MAX')
    # df['month_year_depth'] = df['month_year_depth'].str.replace('CU', 'MAX')

    # Split the 'month_year_depth' column into parts
    df[['prefix', 'month_year', 'depth_cat']] = df['month_year_depth'].str.extract(r'^(PD)-([A-Za-z]{3}-\d{2})-(.+)$')

    # Drop 'prefix' if not needed
    df.drop(columns='prefix', inplace=True)


In [ ]:
Gemeinde_2080_100_26_PD_long.head()

In [ ]:
# Define your DataFrames with their corresponding years
year_df_map = {
    2020: Gemeinde_2020_100_26_PD_long,
    2030: Gemeinde_2030_100_26_PD_long,
    2050: Gemeinde_2050_100_26_PD_long,
    2080: Gemeinde_2080_100_26_PD_long
}

# Loop through each DataFrame
for year, df in year_df_map.items():

    # Filter to only Jan-25
    jan25_df = df[df['month_year'] == 'Jan-25'].reset_index(drop=True)

    # Set the new global variable
    globals()[f'Gemeinde_{year}_100_26_PD_Jan25'] = jan25_df


In [ ]:
# Second part: group and create *_settle DataFrames
for year in [2020, 2030, 2050, 2080]:
    jan25_var_name = f'Gemeinde_{year}_100_26_PD_Jan25'

    if jan25_var_name in globals():
        jan25_df = globals()[jan25_var_name]

        # Group and sum by 'id' and 'month_year'
        settle_df = jan25_df.groupby(['id', 'month_year'], as_index=False)['PD_area'].sum()
        settle_df.columns = ['id', 'month_year', 'settle_area']
        settle_df['settle_area'] = settle_df['settle_area'].astype(float)

        # Save to a new global variable
        globals()[f'{jan25_var_name}_settle'] = settle_df

In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    long_df = globals()[f'Gemeinde_{y}_100_26_PD_Jan25']
    settle_df = globals()[f'Gemeinde_{y}_100_26_PD_Jan25_settle']

    merged_df = pd.merge(long_df, settle_df, on=['id', 'month_year'], how='left')

    globals()[f'Gemeinde_{y}_100_26_PD_Jan25'] = merged_df


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    df = globals()[f'Gemeinde_{y}_100_26_PD_Jan25']
    globals()[f'Gemeinde_{y}_100_26_PD_Jan25'] = df
    df['settle_area_non_zero'] = df.settle_area.replace(0, 1e-6)
    globals()[f'Gemeinde_{y}_100_26_PD_Jan25']['settle_area_non_zero'] = df.settle_area_non_zero
    globals()[f'Gemeinde_{y}_100_26_PD_Jan25']['depth_cat_area_pct'] = (df.PD_area / df.settle_area_non_zero) * 100

In [ ]:
cols = ['id', 'Gemeinde', 'Kreis', 'Land']
years = [2030, 2050, 2080]

base_df = Gemeinde_2020_100_26_PD_Jan25

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_26_PD_Jan25']

    # Direct comparison of the column values (row by row, in order)
    is_same = base_df[cols].equals(current_df[cols])

    if is_same:
        print(f"✅ Columns match exactly for year {year}")
    else:
        print(f"❌ Columns differ for year {year}")


In [ ]:
years = [2030, 2050, 2080]

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_26_PD_Jan25']
    base_df = Gemeinde_2020_100_26_PD_Jan25
    base_df[f'diff_{year}'] = current_df.PD_area - base_df.PD_area

In [ ]:
summary = Gemeinde_2020_100_26_PD_Jan25[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates().copy()
years = [2030, 2050, 2080]

for year in years:
    diff_col = f'diff_{year}'

    pivot = (
        Gemeinde_2020_100_26_PD_Jan25
        .groupby(['id', 'depth_cat'])[diff_col]
        .sum()
        .unstack(fill_value=0)
        .add_prefix(f'sum_diff_{year}_')  # e.g., sum_diff_2030_lt0.5
        .reset_index()
    )

    summary = summary.merge(pivot, on='id', how='left')


In [ ]:
# Convert all newly added columns to float
new_cols = [col for col in summary.columns if col.startswith('sum_diff_')]
summary[new_cols] = summary[new_cols].astype(float)

In [ ]:
summary_26 = summary.copy()

In [ ]:
years = [2030, 2050, 2080]

for year in years:
    summary_26[f'high_risk_{year}'] = (
        summary_26.get(f'sum_diff_{year}_lt0.5', 0) +
        summary_26.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_26.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_26[f'very_high_risk_{year}'] = (
        summary_26.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_26.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_26[f'extreme_risk_{year}'] = summary_26.get(f'sum_diff_{year}_gt1.5', 0)

In [ ]:
summary_26.columns

In [ ]:
value_vars = [
    'high_risk_2030', 'very_high_risk_2030', 'extreme_risk_2030',
    'high_risk_2050', 'very_high_risk_2050', 'extreme_risk_2050',
    'high_risk_2080', 'very_high_risk_2080', 'extreme_risk_2080'
]

pd_summary_26_long = summary_26.melt(
    id_vars=['id', 'Gemeinde', 'Kreis', 'Land'],  # keep these
    value_vars=value_vars,
    var_name='risk_year',
    value_name='pd_diff_2020'
)

# Split the combined column into risk_level and year
pd_summary_26_long[['risk_level', 'year']] = pd_summary_26_long['risk_year'].str.extract(r'(.*)_(\d{4})')
pd_summary_26_long['year'] = pd_summary_26_long['year'].astype(int)


In [ ]:
pd_summary_26_long.head()

### Please store the pre-processed files as pickle files, since you’ll need to load them directly later—especially if the session needs to be restarted due to running out of RAM.

In [ ]:
pd_summary_26_long.to_pickle("pd_summary_26_long.pkl")

In [ ]:
with open("pd_summary_26_long.pkl", "rb") as f:
    pd_summary_26_long = pickle.load(f)

# SSP2-RCP4.5 (Moderate Optimistic)

### Read files, pre-processing and store

In [ ]:
!unzip "/content/drive/MyDrive/Germany_Flood_Study/Germany_flood_study_raw_csv/moderate_optimistic.zip"

In [ ]:
file_paths = [
    # '/content/Germany_Gemeinde_2020_100.xlsx',
    '/content/moderate_optimistic/Germany_Gemeinde_2030_100_45.xlsx',
    '/content/moderate_optimistic/Germany_Gemeinde_2050_100_45.xlsx',
    '/content/moderate_optimistic/Germany_Gemeinde_2080_100_45.xlsx',
]

dataframes = []
for path in file_paths:
    df = pd.read_excel(path)
    df = df.round(6)
    dataframes.append(df)

# Optionally unpack into variables
(Gemeinde_2030_100_45_raw, Gemeinde_2050_100_45_raw, Gemeinde_2080_100_45_raw) = dataframes

In [ ]:
# Date range: every 6 months
dates = pd.date_range(start='2016-07-01', end='2025-01-01', freq='6MS')

# Prefixes and suffixes
prefixes = ['CU', 'FU', 'PD', 'MAX']
suffixes = ['P0', 'lt0.15', 'lt0.5', 'lt1.5', 'gt1.5']

# Correct order: by prefix, then suffix, then date
all_columns = [
    f"{prefix}-{date.strftime('%b-%y')}-{suffix}"
    for prefix in prefixes
    for suffix in suffixes
    for date in dates
]

In [ ]:
# Check if number of replacement columns matches the shape
for df in [Gemeinde_2030_100_45_raw, Gemeinde_2050_100_45_raw, Gemeinde_2080_100_45_raw]:
    if len(all_columns) == df.shape[1] - 3:
      df.columns = list(df.columns[:3]) + all_columns
      df.reset_index(drop=True, inplace=True)
      df['id'] = df.index
    else:
      raise ValueError("Length of generated column names does not match number of columns (excluding the first one).")


In [ ]:
Gemeinde_2080_100_45_raw.head()

In [ ]:
Gemeinde_2030_100_45_raw = Gemeinde_2030_100_45_raw[1:]
Gemeinde_2050_100_45_raw = Gemeinde_2050_100_45_raw[1:]
Gemeinde_2080_100_45_raw = Gemeinde_2080_100_45_raw[1:]

In [ ]:
save_path = "/content/"

# Make sure the directory exists

os.makedirs(save_path, exist_ok=True)

gemeinde_dfs = [Gemeinde_2020_100_raw, Gemeinde_2030_100_45_raw, Gemeinde_2050_100_45_raw, Gemeinde_2080_100_45_raw]

filenames = [
    "Gemeinde_2020_100_raw.pkl", "Gemeinde_2030_100_45_raw.pkl", "Gemeinde_2050_100_45_raw.pkl", "Gemeinde_2080_100_45_raw.pkl"
]

for df, filename in zip(gemeinde_dfs, filenames):
    df.to_pickle(os.path.join(save_path, filename))


### **PLEASE re-run if the session restarts before you finish processing this climate scenario:** Read and load files from here

In [ ]:
# Define the path where the files are stored
save_path = "/content/"

# List of filenames
filenames = [
    "Gemeinde_2020_100_raw.pkl", "Gemeinde_2030_100_45_raw.pkl", "Gemeinde_2050_100_45_raw.pkl", "Gemeinde_2080_100_45_raw.pkl"
]

# Load all DataFrames into a list
gemeinde_dfs = [pd.read_pickle(os.path.join(save_path, filename)) for filename in filenames]

cols_to_convert = [col for col in gemeinde_dfs[0].columns if col not in ['Gemeinde', 'Kreis', 'Land', 'id']]

def convert_and_round(df):
    df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce').round(6)
    return df

gemeinde_dfs = [convert_and_round(df) for df in gemeinde_dfs]

# Optionally assign them back to individual variable names (if needed)
(
  Gemeinde_2020_100_raw,
  Gemeinde_2030_100_45_raw,
  Gemeinde_2050_100_45_raw,
  Gemeinde_2080_100_45_raw
) = gemeinde_dfs


In [ ]:
# Date range: every 6 months
dates = pd.date_range(start='2016-07-01', end='2025-01-01', freq='6MS')

# Prefixes and suffixes
prefixes = ['CU', 'FU', 'PD', 'MAX']
suffixes = ['P0', 'lt0.15', 'lt0.5', 'lt1.5', 'gt1.5']

# Correct order: by prefix, then suffix, then date
all_columns = [
    f"{prefix}-{date.strftime('%b-%y')}-{suffix}"
    for prefix in prefixes
    for suffix in suffixes
    for date in dates
]

## Combinded flooding (max)

### Data preparation

In [ ]:
# List of years to process
years = [2020, 2030, 2050, 2080]

# Columns you want to select (CU columns and Land)
max_columns = ['id', 'Gemeinde', 'Kreis', 'Land'] + [col for col in all_columns if col.startswith('MAX') or col == 'Land']

# Loop through each year and modify the corresponding DataFrame
for year in years:
    if year == 2020:
        df_name = f'Gemeinde_{year}_100_raw'  # 2020 uses '_100_raw'
    else:
        df_name = f'Gemeinde_{year}_100_45_raw'  # 2030, 2050, and 2080 use '_100_85_raw'

    # Check if the DataFrame exists before proceeding
    if df_name in globals():
        # Get the raw DataFrame and select the relevant columns for max flooding data
        df_raw = globals()[df_name]
        globals()[f'Gemeinde_{year}_100_45_MAX'] = df_raw[max_columns]
    else:
        print(f'{df_name} does not exist.')


In [ ]:
len(Gemeinde_2020_100_45_MAX) == len(Gemeinde_2030_100_45_MAX)

In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    globals()[f'Gemeinde_{y}_100_45_MAX_long'] = globals()[f'Gemeinde_{y}_100_45_MAX'].melt(
        id_vars=['id', 'Gemeinde', 'Kreis', 'Land'], var_name='month_year_depth', value_name='MAX_area'
    )


In [ ]:
Gemeinde_2020_100_45_MAX_long.head()

In [ ]:
for df in [Gemeinde_2020_100_45_MAX_long, Gemeinde_2030_100_45_MAX_long, Gemeinde_2050_100_45_MAX_long, Gemeinde_2080_100_45_MAX_long]:

    # Replace 'CU' with 'MAX' if needed (optional step if not already 'MAX')
    # df['month_year_depth'] = df['month_year_depth'].str.replace('CU', 'MAX')

    # Split the 'month_year_depth' column into parts
    df[['prefix', 'month_year', 'depth_cat']] = df['month_year_depth'].str.extract(r'^(MAX)-([A-Za-z]{3}-\d{2})-(.+)$')

    # Drop 'prefix' if not needed
    df.drop(columns='prefix', inplace=True)


In [ ]:
Gemeinde_2080_100_45_MAX_long.head()

In [ ]:
# Define your DataFrames with their corresponding years
year_df_map = {
    2020: Gemeinde_2020_100_45_MAX_long,
    2030: Gemeinde_2030_100_45_MAX_long,
    2050: Gemeinde_2050_100_45_MAX_long,
    2080: Gemeinde_2080_100_45_MAX_long
}

# Loop through each DataFrame
for year, df in year_df_map.items():

    # Filter to only Jan-25
    jan25_df = df[df['month_year'] == 'Jan-25'].reset_index(drop=True)

    # Set the new global variable
    globals()[f'Gemeinde_{year}_100_45_MAX_Jan25'] = jan25_df


In [ ]:
# Second part: group and create *_settle DataFrames
for year in [2020, 2030, 2050, 2080]:
    jan25_var_name = f'Gemeinde_{year}_100_45_MAX_Jan25'

    if jan25_var_name in globals():
        jan25_df = globals()[jan25_var_name]

        # Group and sum by 'id' and 'month_year'
        settle_df = jan25_df.groupby(['id', 'month_year'], as_index=False)['MAX_area'].sum()
        settle_df.columns = ['id', 'month_year', 'settle_area']
        settle_df['settle_area'] = settle_df['settle_area'].astype(float)

        # Save to a new global variable
        globals()[f'{jan25_var_name}_settle'] = settle_df

In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    long_df = globals()[f'Gemeinde_{y}_100_45_MAX_Jan25']
    settle_df = globals()[f'Gemeinde_{y}_100_45_MAX_Jan25_settle']

    merged_df = pd.merge(long_df, settle_df, on=['id', 'month_year'], how='left')

    globals()[f'Gemeinde_{y}_100_45_MAX_Jan25'] = merged_df


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    df = globals()[f'Gemeinde_{y}_100_45_MAX_Jan25']
    globals()[f'Gemeinde_{y}_100_45_MAX_Jan25'] = df
    df['settle_area_non_zero'] = df.settle_area.replace(0, 1e-6)
    globals()[f'Gemeinde_{y}_100_45_MAX_Jan25']['settle_area_non_zero'] = df.settle_area_non_zero
    globals()[f'Gemeinde_{y}_100_45_MAX_Jan25']['depth_cat_area_pct'] = (df.MAX_area / df.settle_area_non_zero) * 100

In [ ]:
years = [2030, 2050, 2080]

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_45_MAX_Jan25']
    base_df = Gemeinde_2020_100_45_MAX_Jan25
    base_df[f'diff_{year}'] = current_df.MAX_area - base_df.MAX_area

In [ ]:
summary = Gemeinde_2020_100_45_MAX_Jan25[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates().copy()
years = [2030, 2050, 2080]

for year in years:
    diff_col = f'diff_{year}'

    pivot = (
        Gemeinde_2020_100_45_MAX_Jan25
        .groupby(['id', 'depth_cat'])[diff_col]
        .sum()
        .unstack(fill_value=0)
        .add_prefix(f'sum_diff_{year}_')  # e.g., sum_diff_2030_lt0.5
        .reset_index()
    )

    summary = summary.merge(pivot, on='id', how='left')


In [ ]:
# Convert all newly added columns to float
new_cols = [col for col in summary.columns if col.startswith('sum_diff_')]
summary[new_cols] = summary[new_cols].astype(float)

In [ ]:
Gemeinde_2020_100_45_MAX_Jan25[Gemeinde_2020_100_45_MAX_Jan25.Gemeinde == 'Allmendingen']

In [ ]:
summary_45 = summary.copy()

In [ ]:
# Step 1: Calculate total MAX_area per id and depth_cat
area_pivot = (
    Gemeinde_2020_100_45_MAX_Jan25
    .groupby(['id', 'depth_cat'])['MAX_area']
    .sum()
    .unstack(fill_value=0)
    .add_prefix('area_')
    .reset_index()
)

# Step 2: Merge into summary_26
summary_45 = summary_45.merge(area_pivot, on='id', how='left')

# Step 3: Calculate percentage risk levels using area for each category
years = [2030, 2050, 2080]
for year in years:
    # Absolute difference sums (already calculated earlier)
    summary_45[f'high_risk_{year}'] = (
        summary_45.get(f'sum_diff_{year}_lt0.5', 0) +
        summary_45.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_45.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_45[f'very_high_risk_{year}'] = (
        summary_45.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_45.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_45[f'extreme_risk_{year}'] = summary_45.get(f'sum_diff_{year}_gt1.5', 0)

    # Denominators from MAX_area sums
    total_area = (
        summary_45.get('area_lt0.5', 0) +
        summary_45.get('area_lt1.5', 0) +
        summary_45.get('area_gt1.5', 0)
    ).replace(0, 1e-6)

    very_high_area = (
        summary_45.get('area_lt1.5', 0) +
        summary_45.get('area_gt1.5', 0)
    ).replace(0, 1e-6)

    extreme_area = summary_45.get('area_gt1.5', 0).replace(0, 1e-6)

    # Percentage risk levels
    summary_45[f'pct_high_risk_{year}'] = (
        summary_45[f'high_risk_{year}'] / total_area * 100
    ).round(2)

    summary_45[f'pct_very_high_risk_{year}'] = (
        summary_45[f'very_high_risk_{year}'] / very_high_area * 100
    ).round(2)

    summary_45[f'pct_extreme_risk_{year}'] = (
        summary_45[f'extreme_risk_{year}'] / extreme_area * 100
    ).round(2)


In [ ]:
summary_45.columns

In [ ]:
value_vars = [
    'high_risk_2030', 'very_high_risk_2030', 'extreme_risk_2030',
    'high_risk_2050', 'very_high_risk_2050', 'extreme_risk_2050',
    'high_risk_2080', 'very_high_risk_2080', 'extreme_risk_2080'
]

# 1. Melt the absolute values
summary_diff_long = summary_45.melt(
    id_vars=['id', 'Gemeinde', 'Kreis', 'Land'],
    value_vars=value_vars,
    var_name='risk_year',
    value_name='max_diff_2020'
)

# 2. Melt the percent difference columns
summary_pct_long = summary_45.melt(
    id_vars=['id', 'Gemeinde', 'Kreis', 'Land'],
    value_vars=[f'pct_{v}' for v in value_vars],
    var_name='risk_year',
    value_name='max_pct_diff_2020'
)

# 3. Clean `risk_year` for both (they must match for merge)
summary_pct_long['risk_year'] = summary_pct_long['risk_year'].str.replace('pct_', '')

# 4. Merge the two melted DataFrames
summary_45_long = pd.merge(
    summary_diff_long,
    summary_pct_long,
    on=['id', 'Gemeinde', 'Kreis', 'Land', 'risk_year']
)

# 5. Extract risk level and year
summary_45_long[['risk_level', 'year']] = summary_45_long['risk_year'].str.extract(r'(.*)_(\d{4})')
summary_45_long['year'] = summary_45_long['year'].astype(int)


### Include GEM_ID to match with the gemeinden geometries for mapping

In [ ]:
with open("max_summary_26_long.pkl", "rb") as f:
    summary_26_long = pickle.load(f)

In [ ]:
# clean comparison of df1 and df2 by 'id'

# 1. select and sort consistently
df1_clean = summary_45_long[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates()
df2_clean = summary_26_long[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates()

# 2. drop non-Gemeinde entries (e.g., rows where Gemeinde or Land is missing)
df1_clean = df1_clean.dropna(subset=['Gemeinde', 'Land'])
df2_clean = df2_clean.dropna(subset=['Gemeinde', 'Land'])

# 3. set 'id' as index to ensure alignment by identifier, not row order
df1_clean = df1_clean.set_index('id').sort_index()
df2_clean = df2_clean.set_index('id').sort_index()

# 4. align and find differences
# keep only ids present in both datasets
common_ids = df1_clean.index.intersection(df2_clean.index)

# find true mismatches by comparing all columns for each id
diff_mask = ~(df1_clean.loc[common_ids] == df2_clean.loc[common_ids]).all(axis=1)
mismatched_rows = pd.concat(
    [df1_clean.loc[diff_mask], df2_clean.loc[diff_mask]],
    keys=['df1', 'df2']
)

# 5. print results
print(f"number of mismatched ids: {diff_mask.sum()}")
display(mismatched_rows)

# please note that these results are expected and not true mismatches:
# the two "Helgoland" entries in Pinneberg, Schleswig-holstein were renamed with suffixes "-1" and "-2"
# to enable joining with the geometries.
# since there are no settlement areas in these two gemeinden, the results and maps are not affected.

In [ ]:
summary_45_long = summary_45_long.merge(summary_26_long[['id', 'GEM_ID']].drop_duplicates().copy(), on='id', how='left')

In [ ]:
summary_45_long.GEM_ID.nunique()

### Please store the pre-processed files as pickle files, since you’ll need to load them directly later—especially if the session needs to be restarted due to running out of RAM.

In [ ]:
summary_45_long.to_pickle("max_summary_45_long.pkl")

In [ ]:
with open("max_summary_45_long.pkl", "rb") as f:
    summary_45_long = pickle.load(f)

### Flooding exposure difference map (future years vs. 2020) - prep

In [ ]:
for year in [2030, 2050, 2080]:
    globals()[f"summary_45_above_015_{year}"] = summary_45_long[
        (summary_45_long['risk_level'] == 'high_risk') &
        (summary_45_long['Gemeinde'] != 'Germany') &
        (summary_45_long['year'] == year)
    ].copy()


In [ ]:
admin3_boundary = gpd.read_file('/content/admin-GIS/vg250_01-01.gk3.shape.ebenen/vg250_ebenen_0101/VG250_GEM.shp')

In [ ]:
years = [2030, 2050, 2080]

for y in years:
    summary_45_above_015_df = globals()[f'summary_45_above_015_{y}']

    merged_gdf = admin3_boundary.merge(summary_45_above_015_df, left_on='OBJID', right_on='GEM_ID', how='right')

    globals()[f'summary_45_above_015_{y}_gdf'] = merged_gdf

In [ ]:
years = [2030, 2050, 2080]

for y in years:
    summary_45_above_015_gdf = globals()[f'summary_45_above_015_{y}_gdf']
    print(f"Number of null geometries in summary_45_above_015_{y}_gdf: {len(summary_45_above_015_gdf[summary_45_above_015_gdf.geometry.isna()])}")
    print(f"Number of unique geometries in summary_45_above_015_{y}_gdf: {len(summary_45_above_015_gdf.geometry.unique())}")

In [ ]:
# Summary statistics for table in appendix
years = [2030, 2050, 2080]
results = {}

for year in years:
    df_name = f"summary_45_above_015_{year}_gdf"
    df = globals()[df_name]   # fetch DataFrame by name

    positive_sum = df.loc[df.max_diff_2020 > 0].max_diff_2020.sum().round(2)
    negative_sum = df.loc[df.max_diff_2020 < 0].max_diff_2020.sum().round(2)

    results[year] = {"positive_sum": positive_sum, "negative_sum": negative_sum}

results


### Percentage difference map

In [ ]:
# Years to plot
years = [2030, 2050, 2080]
save_path = "/content/drive/MyDrive/Germany_Flood_Study/Gemeinde_climate_scenario_maps/final_adjustments"
os.makedirs(save_path, exist_ok=True)

for y in years:
    gdf = globals()[f'summary_45_above_015_{y}_gdf'].copy()
    gdf = gdf.to_crs(epsg=3857)

    # Mask 0 values for transparency
    gdf['masked_pct_diff'] = gdf['max_pct_diff_2020'].replace(0, np.nan)

    # Colors & breaks
    custom_colors = [
        '#40e0d0',  # < -10%
        '#00ffff',  # -10% ~ -1%
        '#808080',  # -1% ~ 1%
        '#fee5d9',  # 1% ~ 10%
        '#fb6a4a',  # 10% ~ 50%
        '#de2d26',  # 50% ~ 100%
        '#a50f15'   # > 100%
    ]
    custom_breaks = [-np.inf, -10, -1, 1, 10, 50, 100, np.inf]

    cmap = ListedColormap(custom_colors)
    norm = BoundaryNorm(custom_breaks, ncolors=len(custom_colors), clip=False)

    # Create figure and axis INSIDE loop
    fig, ax = plt.subplots(figsize=(14, 12))

    # Plot
    gdf.plot(
        ax=ax,
        column='masked_pct_diff',
        cmap=cmap,
        norm=norm,
        edgecolor=None,
        linewidth=0,
        legend=False
    )

    # Overlay geometry boundaries
    gdf.boundary.plot(ax=ax, color='grey', linewidth=0.05)

    # Basemap
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.PositronNoLabels, crs=gdf.crs)

    # Cleanup
    ax.axis('off')

    # North arrow & scale bar
    add_north_arrow(ax, scale=0.75, xlim_pos=0.9025, ylim_pos=0.9,
                    color='black', text_scaler=4, text_yT=-1.25)
    scale1 = ScaleBar(dx=1, location="lower right", scale_loc="bottom")
    ax.add_artist(scale1)

    # Custom legend
    legend_labels = [
        "<-10%", "-10% ~ -1%",
        "-1% ~ 1%",
        "1% ~ 10%", "10% ~ 50%", "50% ~ 100%", ">100%"
    ]
    circles = [
        Line2D([0], [0], marker='o', color='None',
              markerfacecolor=custom_colors[i],
              markeredgecolor='None',
              markersize=10,
              label=legend_labels[i])
        for i in range(len(legend_labels))
    ]
    ax.legend(
        handles=circles,
        title="Exposure change",
        loc='upper left',
        fontsize=12,
        title_fontsize=14
    )

    # Save
    file_name = f"gemeinde_exposure_pct_change_high_risk_45_{y}yr_separate_dry_wet_cyansv2.png"
    full_path = os.path.join(save_path, file_name)
    plt.tight_layout()
    # plt.savefig(full_path, dpi=300, bbox_inches='tight')
    plt.show()
    # plt.close()


## Coastal flooding undefended

### Data Preparation

In [ ]:
# List of years to process
years = [2020, 2030, 2050, 2080]

# Columns you want to select
cu_columns = ['id', 'Gemeinde', 'Kreis', 'Land'] + [col for col in all_columns if col.startswith('CU')]

# Loop through each year and modify the corresponding DataFrame
for year in years:
    if year == 2020:
        df_name = f'Gemeinde_{year}_100_raw'  # 2020 uses '_100_raw'
    else:
        df_name = f'Gemeinde_{year}_100_45_raw'  # 2030, 2050, and 2080 use '_100_85_raw'

    # Check if the DataFrame exists before proceeding
    if df_name in globals():
        # Get the raw DataFrame and select the relevant columns for max flooding data
        df_raw = globals()[df_name]
        globals()[f'Gemeinde_{year}_100_45_CU'] = df_raw[cu_columns]
    else:
        print(f'{df_name} does not exist.')


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    globals()[f'Gemeinde_{y}_100_45_CU_long'] = globals()[f'Gemeinde_{y}_100_45_CU'].melt(
        id_vars=['id', 'Gemeinde', 'Kreis', 'Land'], var_name='month_year_depth', value_name='CU_area'
    )


In [ ]:
Gemeinde_2020_100_45_CU_long.head()

In [ ]:
for df in [Gemeinde_2020_100_45_CU_long, Gemeinde_2030_100_45_CU_long, Gemeinde_2050_100_45_CU_long, Gemeinde_2080_100_45_CU_long]:

    # Split the 'month_year_depth' column into parts
    df[['prefix', 'month_year', 'depth_cat']] = df['month_year_depth'].str.extract(r'^(CU)-([A-Za-z]{3}-\d{2})-(.+)$')

    # Drop 'prefix' if not needed
    df.drop(columns='prefix', inplace=True)


In [ ]:
Gemeinde_2080_100_45_CU_long.head()

In [ ]:
# Define your DataFrames with their corresponding years
year_df_map = {
    2020: Gemeinde_2020_100_45_CU_long,
    2030: Gemeinde_2030_100_45_CU_long,
    2050: Gemeinde_2050_100_45_CU_long,
    2080: Gemeinde_2080_100_45_CU_long
}

# Loop through each DataFrame
for year, df in year_df_map.items():

    # Filter to only Jan-25
    jan25_df = df[df['month_year'] == 'Jan-25'].reset_index(drop=True)

    # Set the new global variable
    globals()[f'Gemeinde_{year}_100_45_CU_Jan25'] = jan25_df


In [ ]:
# Second part: group and create *_settle DataFrames
for year in [2020, 2030, 2050, 2080]:
    jan25_var_name = f'Gemeinde_{year}_100_45_CU_Jan25'

    if jan25_var_name in globals():
        jan25_df = globals()[jan25_var_name]

        # Group and sum by 'id' and 'month_year'
        settle_df = jan25_df.groupby(['id', 'month_year'], as_index=False)['CU_area'].sum()
        settle_df.columns = ['id', 'month_year', 'settle_area']
        settle_df['settle_area'] = settle_df['settle_area'].astype(float)

        # Save to a new global variable
        globals()[f'{jan25_var_name}_settle'] = settle_df

In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    long_df = globals()[f'Gemeinde_{y}_100_45_CU_Jan25']
    settle_df = globals()[f'Gemeinde_{y}_100_45_CU_Jan25_settle']

    merged_df = pd.merge(long_df, settle_df, on=['id', 'month_year'], how='left')

    globals()[f'Gemeinde_{y}_100_45_CU_Jan25'] = merged_df


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    df = globals()[f'Gemeinde_{y}_100_45_CU_Jan25']
    globals()[f'Gemeinde_{y}_100_45_CU_Jan25'] = df
    df['settle_area_non_zero'] = df.settle_area.replace(0, 1e-6)
    globals()[f'Gemeinde_{y}_100_45_CU_Jan25']['settle_area_non_zero'] = df.settle_area_non_zero
    globals()[f'Gemeinde_{y}_100_45_CU_Jan25']['depth_cat_area_pct'] = (df.CU_area / df.settle_area_non_zero) * 100

In [ ]:
cols = ['id', 'Gemeinde', 'Kreis', 'Land']
years = [2030, 2050, 2080]

base_df = Gemeinde_2020_100_45_CU_Jan25

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_45_CU_Jan25']

    # Direct comparison of the column values (row by row, in order)
    is_same = base_df[cols].equals(current_df[cols])

    if is_same:
        print(f"✅ Columns match exactly for year {year}")
    else:
        print(f"❌ Columns differ for year {year}")


In [ ]:
years = [2030, 2050, 2080]

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_45_CU_Jan25']
    base_df = Gemeinde_2020_100_45_CU_Jan25
    base_df[f'diff_{year}'] = current_df.CU_area - base_df.CU_area

In [ ]:
summary = Gemeinde_2020_100_45_CU_Jan25[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates().copy()
years = [2030, 2050, 2080]

for year in years:
    diff_col = f'diff_{year}'

    pivot = (
        Gemeinde_2020_100_45_CU_Jan25
        .groupby(['id', 'depth_cat'])[diff_col]
        .sum()
        .unstack(fill_value=0)
        .add_prefix(f'sum_diff_{year}_')  # e.g., sum_diff_2030_lt0.5
        .reset_index()
    )

    summary = summary.merge(pivot, on='id', how='left')


In [ ]:
# Convert all newly added columns to float
new_cols = [col for col in summary.columns if col.startswith('sum_diff_')]
summary[new_cols] = summary[new_cols].astype(float)

In [ ]:
Gemeinde_2020_100_45_CU_Jan25[Gemeinde_2020_100_45_CU_Jan25.Gemeinde == 'Allmendingen']

In [ ]:
summary_45 = summary.copy()

In [ ]:
years = [2030, 2050, 2080]

for year in years:
    summary_45[f'high_risk_{year}'] = (
        summary_45.get(f'sum_diff_{year}_lt0.5', 0) +
        summary_45.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_45.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_45[f'very_high_risk_{year}'] = (
        summary_45.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_45.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_45[f'extreme_risk_{year}'] = summary_45.get(f'sum_diff_{year}_gt1.5', 0)

In [ ]:
summary_45.columns

In [ ]:
value_vars = [
    'high_risk_2030', 'very_high_risk_2030', 'extreme_risk_2030',
    'high_risk_2050', 'very_high_risk_2050', 'extreme_risk_2050',
    'high_risk_2080', 'very_high_risk_2080', 'extreme_risk_2080'
]

cu_summary_45_long = summary_45.melt(
    id_vars=['id', 'Gemeinde', 'Kreis', 'Land'],  # keep these
    value_vars=value_vars,
    var_name='risk_year',
    value_name='cu_diff_2020'
)

# Split the combined column into risk_level and year
cu_summary_45_long[['risk_level', 'year']] = cu_summary_45_long['risk_year'].str.extract(r'(.*)_(\d{4})')
cu_summary_45_long['year'] = cu_summary_45_long['year'].astype(int)


### Please store the pre-processed files as pickle files, since you’ll need to load them directly later—especially if the session needs to be restarted due to running out of RAM.

In [ ]:
cu_summary_45_long.to_pickle("cu_summary_45_long.pkl")

In [ ]:
with open('cu_summary_45_long.pkl', 'rb') as f:
    cu_summary_45_long = pickle.load(f)

## Fluvial flooding undefended

### Data Preparation

In [ ]:
# List of years to process
years = [2020, 2030, 2050, 2080]

# Columns you want to select (CU columns and Land)
fu_columns = ['id', 'Gemeinde', 'Kreis', 'Land'] + [col for col in all_columns]

# Loop through each year and modify the corresponding DataFrame
for year in years:
    if year == 2020:
        df_name = f'Gemeinde_{year}_100_raw'  # 2020 uses '_100_raw'
    else:
        df_name = f'Gemeinde_{year}_100_45_raw'  # 2030, 2050, and 2080 use '_100_85_raw'

    # Check if the DataFrame exists before proceeding
    if df_name in globals():
        # Get the raw DataFrame and select the relevant columns for max flooding data
        df_raw = globals()[df_name]
        globals()[f'Gemeinde_{year}_100_45_FU'] = df_raw[fu_columns]
    else:
        print(f'{df_name} does not exist.')


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    globals()[f'Gemeinde_{y}_100_45_FU_long'] = globals()[f'Gemeinde_{y}_100_45_FU'].melt(
        id_vars=['id', 'Gemeinde', 'Kreis', 'Land'], var_name='month_year_depth', value_name='FU_area'
    )


In [ ]:
Gemeinde_2020_100_45_FU_long.head()

In [ ]:
for df in [Gemeinde_2020_100_45_FU_long, Gemeinde_2030_100_45_FU_long, Gemeinde_2050_100_45_FU_long, Gemeinde_2080_100_45_FU_long]:

    # Split the 'month_year_depth' column into parts
    df[['prefix', 'month_year', 'depth_cat']] = df['month_year_depth'].str.extract(r'^(FU)-([A-Za-z]{3}-\d{2})-(.+)$')

    # Drop 'prefix' if not needed
    df.drop(columns='prefix', inplace=True)


In [ ]:
Gemeinde_2080_100_45_FU_long.head()

In [ ]:
# Define your DataFrames with their corresponding years
year_df_map = {
    2020: Gemeinde_2020_100_45_FU_long,
    2030: Gemeinde_2030_100_45_FU_long,
    2050: Gemeinde_2050_100_45_FU_long,
    2080: Gemeinde_2080_100_45_FU_long
}

# Loop through each DataFrame
for year, df in year_df_map.items():

    # Filter to only Jan-25
    jan25_df = df[df['month_year'] == 'Jan-25'].reset_index(drop=True)

    # Set the new global variable
    globals()[f'Gemeinde_{year}_100_45_FU_Jan25'] = jan25_df


In [ ]:
# Second part: group and create *_settle DataFrames
for year in [2020, 2030, 2050, 2080]:
    jan25_var_name = f'Gemeinde_{year}_100_45_FU_Jan25'

    if jan25_var_name in globals():
        jan25_df = globals()[jan25_var_name]

        # Group and sum by 'id' and 'month_year'
        settle_df = jan25_df.groupby(['id', 'month_year'], as_index=False)['FU_area'].sum()
        settle_df.columns = ['id', 'month_year', 'settle_area']
        settle_df['settle_area'] = settle_df['settle_area'].astype(float)

        # Save to a new global variable
        globals()[f'{jan25_var_name}_settle'] = settle_df

In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    long_df = globals()[f'Gemeinde_{y}_100_45_FU_Jan25']
    settle_df = globals()[f'Gemeinde_{y}_100_45_FU_Jan25_settle']

    merged_df = pd.merge(long_df, settle_df, on=['id', 'month_year'], how='left')

    globals()[f'Gemeinde_{y}_100_45_FU_Jan25'] = merged_df


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    df = globals()[f'Gemeinde_{y}_100_45_FU_Jan25']
    globals()[f'Gemeinde_{y}_100_45_FU_Jan25'] = df
    df['settle_area_non_zero'] = df.settle_area.replace(0, 1e-6)
    globals()[f'Gemeinde_{y}_100_45_FU_Jan25']['settle_area_non_zero'] = df.settle_area_non_zero
    globals()[f'Gemeinde_{y}_100_45_FU_Jan25']['depth_cat_area_pct'] = (df.FU_area / df.settle_area_non_zero) * 100

In [ ]:
cols = ['id', 'Gemeinde', 'Kreis', 'Land']
years = [2030, 2050, 2080]

base_df = Gemeinde_2020_100_45_FU_Jan25

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_45_FU_Jan25']

    # Direct comparison of the column values (row by row, in order)
    is_same = base_df[cols].equals(current_df[cols])

    if is_same:
        print(f"✅ Columns match exactly for year {year}")
    else:
        print(f"❌ Columns differ for year {year}")


In [ ]:
years = [2030, 2050, 2080]

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_45_FU_Jan25']
    base_df = Gemeinde_2020_100_45_FU_Jan25
    base_df[f'diff_{year}'] = current_df.FU_area - base_df.FU_area

In [ ]:
summary = Gemeinde_2020_100_45_FU_Jan25[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates().copy()
years = [2030, 2050, 2080]

for year in years:
    diff_col = f'diff_{year}'

    pivot = (
        Gemeinde_2020_100_45_FU_Jan25
        .groupby(['id', 'depth_cat'])[diff_col]
        .sum()
        .unstack(fill_value=0)
        .add_prefix(f'sum_diff_{year}_')  # e.g., sum_diff_2030_lt0.5
        .reset_index()
    )

    summary = summary.merge(pivot, on='id', how='left')


In [ ]:
# Convert all newly added columns to float
new_cols = [col for col in summary.columns if col.startswith('sum_diff_')]
summary[new_cols] = summary[new_cols].astype(float)

In [ ]:
summary_45 = summary.copy()

In [ ]:
years = [2030, 2050, 2080]

for year in years:
    summary_45[f'high_risk_{year}'] = (
        summary_45.get(f'sum_diff_{year}_lt0.5', 0) +
        summary_45.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_45.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_45[f'very_high_risk_{year}'] = (
        summary_45.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_45.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_45[f'extreme_risk_{year}'] = summary_45.get(f'sum_diff_{year}_gt1.5', 0)

In [ ]:
summary_45.columns

In [ ]:
value_vars = [
    'high_risk_2030', 'very_high_risk_2030', 'extreme_risk_2030',
    'high_risk_2050', 'very_high_risk_2050', 'extreme_risk_2050',
    'high_risk_2080', 'very_high_risk_2080', 'extreme_risk_2080'
]

fu_summary_45_long = summary_45.melt(
    id_vars=['id', 'Gemeinde', 'Kreis', 'Land'],  # keep these
    value_vars=value_vars,
    var_name='risk_year',
    value_name='fu_diff_2020'
)

# Split the combined column into risk_level and year
fu_summary_45_long[['risk_level', 'year']] = fu_summary_45_long['risk_year'].str.extract(r'(.*)_(\d{4})')
fu_summary_45_long['year'] = fu_summary_45_long['year'].astype(int)


### Please store the pre-processed files as pickle files, since you’ll need to load them directly later—especially if the session needs to be restarted due to running out of RAM.

In [ ]:
fu_summary_45_long.to_pickle("fu_summary_45_long.pkl")

In [ ]:
with open('fu_summary_45_long.pkl', 'rb') as f:
    fu_summary_45_long = pickle.load(f)

## Pluvial flooding defended

### Data Preparation

In [ ]:
# List of years to process
years = [2020, 2030, 2050, 2080]

# Columns you want to select (CU columns and Land)
pd_columns = ['id', 'Gemeinde', 'Kreis', 'Land'] + [col for col in all_columns if col.startswith('PD')]

# Loop through each year and modify the corresponding DataFrame
for year in years:
    if year == 2020:
        df_name = f'Gemeinde_{year}_100_raw'  # 2020 uses '_100_raw'
    else:
        df_name = f'Gemeinde_{year}_100_45_raw'  # 2030, 2050, and 2080 use '_100_85_raw'

    # Check if the DataFrame exists before proceeding
    if df_name in globals():
        # Get the raw DataFrame and select the relevant columns for max flooding data
        df_raw = globals()[df_name]
        globals()[f'Gemeinde_{year}_100_45_PD'] = df_raw[pd_columns]
    else:
        print(f'{df_name} does not exist.')


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    globals()[f'Gemeinde_{y}_100_45_PD_long'] = globals()[f'Gemeinde_{y}_100_45_PD'].melt(
        id_vars=['id', 'Gemeinde', 'Kreis', 'Land'], var_name='month_year_depth', value_name='PD_area'
    )


In [ ]:
Gemeinde_2020_100_45_PD_long.head()

In [ ]:
for df in [Gemeinde_2020_100_45_PD_long, Gemeinde_2030_100_45_PD_long, Gemeinde_2050_100_45_PD_long, Gemeinde_2080_100_45_PD_long]:

    # Split the 'month_year_depth' column into parts
    df[['prefix', 'month_year', 'depth_cat']] = df['month_year_depth'].str.extract(r'^(PD)-([A-Za-z]{3}-\d{2})-(.+)$')

    # Drop 'prefix' if not needed
    df.drop(columns='prefix', inplace=True)


In [ ]:
Gemeinde_2080_100_45_PD_long.head()

In [ ]:
# Define your DataFrames with their corresponding years
year_df_map = {
    2020: Gemeinde_2020_100_45_PD_long,
    2030: Gemeinde_2030_100_45_PD_long,
    2050: Gemeinde_2050_100_45_PD_long,
    2080: Gemeinde_2080_100_45_PD_long
}

# Loop through each DataFrame
for year, df in year_df_map.items():

    # Filter to only Jan-25
    jan25_df = df[df['month_year'] == 'Jan-25'].reset_index(drop=True)

    # Set the new global variable
    globals()[f'Gemeinde_{year}_100_45_PD_Jan25'] = jan25_df


In [ ]:
# Second part: group and create *_settle DataFrames
for year in [2020, 2030, 2050, 2080]:
    jan25_var_name = f'Gemeinde_{year}_100_45_PD_Jan25'

    if jan25_var_name in globals():
        jan25_df = globals()[jan25_var_name]

        # Group and sum by 'id' and 'month_year'
        settle_df = jan25_df.groupby(['id', 'month_year'], as_index=False)['PD_area'].sum()
        settle_df.columns = ['id', 'month_year', 'settle_area']
        settle_df['settle_area'] = settle_df['settle_area'].astype(float)

        # Save to a new global variable
        globals()[f'{jan25_var_name}_settle'] = settle_df

In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    long_df = globals()[f'Gemeinde_{y}_100_45_PD_Jan25']
    settle_df = globals()[f'Gemeinde_{y}_100_45_PD_Jan25_settle']

    merged_df = pd.merge(long_df, settle_df, on=['id', 'month_year'], how='left')

    globals()[f'Gemeinde_{y}_100_45_PD_Jan25'] = merged_df


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    df = globals()[f'Gemeinde_{y}_100_45_PD_Jan25']
    globals()[f'Gemeinde_{y}_100_45_PD_Jan25'] = df
    df['settle_area_non_zero'] = df.settle_area.replace(0, 1e-6)
    globals()[f'Gemeinde_{y}_100_45_PD_Jan25']['settle_area_non_zero'] = df.settle_area_non_zero
    globals()[f'Gemeinde_{y}_100_45_PD_Jan25']['depth_cat_area_pct'] = (df.PD_area / df.settle_area_non_zero) * 100

In [ ]:
cols = ['id', 'Gemeinde', 'Kreis', 'Land']
years = [2030, 2050, 2080]

base_df = Gemeinde_2020_100_45_PD_Jan25

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_45_PD_Jan25']

    # Direct comparison of the column values (row by row, in order)
    is_same = base_df[cols].equals(current_df[cols])

    if is_same:
        print(f"✅ Columns match exactly for year {year}")
    else:
        print(f"❌ Columns differ for year {year}")


In [ ]:
years = [2030, 2050, 2080]

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_45_PD_Jan25']
    base_df = Gemeinde_2020_100_45_PD_Jan25
    base_df[f'diff_{year}'] = current_df.PD_area - base_df.PD_area

In [ ]:
summary = Gemeinde_2020_100_45_PD_Jan25[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates().copy()
years = [2030, 2050, 2080]

for year in years:
    diff_col = f'diff_{year}'

    pivot = (
        Gemeinde_2020_100_45_PD_Jan25
        .groupby(['id', 'depth_cat'])[diff_col]
        .sum()
        .unstack(fill_value=0)
        .add_prefix(f'sum_diff_{year}_')  # e.g., sum_diff_2030_lt0.5
        .reset_index()
    )

    summary = summary.merge(pivot, on='id', how='left')


In [ ]:
# Convert all newly added columns to float
new_cols = [col for col in summary.columns if col.startswith('sum_diff_')]
summary[new_cols] = summary[new_cols].astype(float)

In [ ]:
summary_45 = summary.copy()

In [ ]:
years = [2030, 2050, 2080]

for year in years:
    summary_45[f'high_risk_{year}'] = (
        summary_45.get(f'sum_diff_{year}_lt0.5', 0) +
        summary_45.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_45.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_45[f'very_high_risk_{year}'] = (
        summary_45.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_45.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_45[f'extreme_risk_{year}'] = summary_45.get(f'sum_diff_{year}_gt1.5', 0)

In [ ]:
summary_45.columns

In [ ]:
value_vars = [
    'high_risk_2030', 'very_high_risk_2030', 'extreme_risk_2030',
    'high_risk_2050', 'very_high_risk_2050', 'extreme_risk_2050',
    'high_risk_2080', 'very_high_risk_2080', 'extreme_risk_2080'
]

pd_summary_45_long = summary_45.melt(
    id_vars=['id', 'Gemeinde', 'Kreis', 'Land'],  # keep these
    value_vars=value_vars,
    var_name='risk_year',
    value_name='pd_diff_2020'
)

# Split the combined column into risk_level and year
pd_summary_45_long[['risk_level', 'year']] = pd_summary_45_long['risk_year'].str.extract(r'(.*)_(\d{4})')
pd_summary_45_long['year'] = pd_summary_45_long['year'].astype(int)


### Please store the pre-processed files as pickle files, since you’ll need to load them directly later—especially if the session needs to be restarted due to running out of RAM.

In [ ]:
pd_summary_45_long.to_pickle("pd_summary_45_long.pkl")

In [ ]:
with open("pd_summary_45_long.pkl", "rb") as f:
    pd_summary_45_long = pickle.load(f)

# SSP3-RCP7.0 (Moderate Pessimistic)

### Read files, pre-processing and store

In [ ]:
!unzip "/content/drive/MyDrive/Germany_Flood_Study/Germany_flood_study_raw_csv/moderate_pessimistic.zip"

In [ ]:
file_paths = [
    # '/content/Germany_Gemeinde_2020_100.xlsx',
    '/content/moderate_pessimistic/Germany_Gemeinde_2030_100_70.xlsx',
    '/content/moderate_pessimistic/Germany_Gemeinde_2050_100_70.xlsx',
    '/content/moderate_pessimistic/Germany_Gemeinde_2080_100_70.xlsx',
]

dataframes = []
for path in file_paths:
    df = pd.read_excel(path)
    df = df.round(6)
    dataframes.append(df)

# Optionally unpack into variables
(Gemeinde_2030_100_70_raw, Gemeinde_2050_100_70_raw, Gemeinde_2080_100_70_raw) = dataframes

In [ ]:
# Date range: every 6 months
dates = pd.date_range(start='2016-07-01', end='2025-01-01', freq='6MS')

# Prefixes and suffixes
prefixes = ['CU', 'FU', 'PD', 'MAX']
suffixes = ['P0', 'lt0.15', 'lt0.5', 'lt1.5', 'gt1.5']

# Correct order: by prefix, then suffix, then date
all_columns = [
    f"{prefix}-{date.strftime('%b-%y')}-{suffix}"
    for prefix in prefixes
    for suffix in suffixes
    for date in dates
]

In [ ]:
# Check if number of replacement columns matches the shape
for df in [Gemeinde_2030_100_70_raw, Gemeinde_2050_100_70_raw, Gemeinde_2080_100_70_raw]:
    if len(all_columns) == df.shape[1] - 3:
      df.columns = list(df.columns[:3]) + all_columns
      df.reset_index(drop=True, inplace=True)
      df['id'] = df.index
    else:
      raise ValueError("Length of generated column names does not match number of columns (excluding the first one).")


In [ ]:
Gemeinde_2030_100_70_raw.head()

In [ ]:
Gemeinde_2030_100_70_raw = Gemeinde_2030_100_70_raw[1:]
Gemeinde_2050_100_70_raw = Gemeinde_2050_100_70_raw[1:]
Gemeinde_2080_100_70_raw = Gemeinde_2080_100_70_raw[1:]

In [ ]:
save_path = "/content/"

# Make sure the directory exists

os.makedirs(save_path, exist_ok=True)

gemeinde_dfs = [Gemeinde_2020_100_raw, Gemeinde_2030_100_70_raw, Gemeinde_2050_100_70_raw, Gemeinde_2080_100_70_raw]

filenames = [
    "Gemeinde_2020_100_raw.pkl", "Gemeinde_2030_100_70_raw.pkl", "Gemeinde_2050_100_70_raw.pkl", "Gemeinde_2080_100_70_raw.pkl"
]

for df, filename in zip(gemeinde_dfs, filenames):
    df.to_pickle(os.path.join(save_path, filename))


### **PLEASE re-run if the session restarts before you finish processing this climate scenario:** Read and load file from here

In [ ]:
# Define the path where the files are stored
save_path = "/content/"

# List of filenames
filenames = [
    "Gemeinde_2020_100_raw.pkl", "Gemeinde_2030_100_70_raw.pkl", "Gemeinde_2050_100_70_raw.pkl", "Gemeinde_2080_100_70_raw.pkl"
]

# Load all DataFrames into a list
gemeinde_dfs = [pd.read_pickle(os.path.join(save_path, filename)) for filename in filenames]

cols_to_convert = [col for col in gemeinde_dfs[0].columns if col not in ['Gemeinde', 'Kreis', 'Land', 'id']]

def convert_and_round(df):
    df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce').round(6)
    return df

gemeinde_dfs = [convert_and_round(df) for df in gemeinde_dfs]

# Optionally assign them back to individual variable names (if needed)
(
  Gemeinde_2020_100_raw,
  Gemeinde_2030_100_70_raw,
  Gemeinde_2050_100_70_raw,
  Gemeinde_2080_100_70_raw
) = gemeinde_dfs


In [ ]:
# Date range: every 6 months
dates = pd.date_range(start='2016-07-01', end='2025-01-01', freq='6MS')

# Prefixes and suffixes
prefixes = ['CU', 'FU', 'PD', 'MAX']
suffixes = ['P0', 'lt0.15', 'lt0.5', 'lt1.5', 'gt1.5']

# Correct order: by prefix, then suffix, then date
all_columns = [
    f"{prefix}-{date.strftime('%b-%y')}-{suffix}"
    for prefix in prefixes
    for suffix in suffixes
    for date in dates
]

## Combined flooding (max)

### Data preparation

In [ ]:
# List of years to process
years = [2020, 2030, 2050, 2080]

# Columns you want to select (CU columns and Land)
max_columns = ['id', 'Gemeinde', 'Kreis', 'Land'] + [col for col in all_columns if col.startswith('MAX') or col == 'Land']

# Loop through each year and modify the corresponding DataFrame
for year in years:
    if year == 2020:
        df_name = f'Gemeinde_{year}_100_raw'  # 2020 uses '_100_raw'
    else:
        df_name = f'Gemeinde_{year}_100_70_raw'  # 2030, 2050, and 2080 use '_100_85_raw'

    # Check if the DataFrame exists before proceeding
    if df_name in globals():
        # Get the raw DataFrame and select the relevant columns for max flooding data
        df_raw = globals()[df_name]
        globals()[f'Gemeinde_{year}_100_70_MAX'] = df_raw[max_columns]
    else:
        print(f'{df_name} does not exist.')


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    globals()[f'Gemeinde_{y}_100_70_MAX_long'] = globals()[f'Gemeinde_{y}_100_70_MAX'].melt(
        id_vars=['id', 'Gemeinde', 'Kreis', 'Land'], var_name='month_year_depth', value_name='MAX_area'
    )


In [ ]:
Gemeinde_2020_100_70_MAX_long.head()

In [ ]:
for df in [Gemeinde_2020_100_70_MAX_long, Gemeinde_2030_100_70_MAX_long, Gemeinde_2050_100_70_MAX_long, Gemeinde_2080_100_70_MAX_long]:

    # Replace 'CU' with 'MAX' if needed (optional step if not already 'MAX')
    # df['month_year_depth'] = df['month_year_depth'].str.replace('CU', 'MAX')

    # Split the 'month_year_depth' column into parts
    df[['prefix', 'month_year', 'depth_cat']] = df['month_year_depth'].str.extract(r'^(MAX)-([A-Za-z]{3}-\d{2})-(.+)$')

    # Drop 'prefix' if not needed
    df.drop(columns='prefix', inplace=True)


In [ ]:
Gemeinde_2080_100_70_MAX_long.head()

In [ ]:
# Define your DataFrames with their corresponding years
year_df_map = {
    2020: Gemeinde_2020_100_70_MAX_long,
    2030: Gemeinde_2030_100_70_MAX_long,
    2050: Gemeinde_2050_100_70_MAX_long,
    2080: Gemeinde_2080_100_70_MAX_long
}

# Loop through each DataFrame
for year, df in year_df_map.items():

    # Filter to only Jan-25
    jan25_df = df[df['month_year'] == 'Jan-25'].reset_index(drop=True)

    # Set the new global variable
    globals()[f'Gemeinde_{year}_100_70_MAX_Jan25'] = jan25_df


In [ ]:
# Second part: group and create *_settle DataFrames
for year in [2020, 2030, 2050, 2080]:
    jan25_var_name = f'Gemeinde_{year}_100_70_MAX_Jan25'

    if jan25_var_name in globals():
        jan25_df = globals()[jan25_var_name]

        # Group and sum by 'id' and 'month_year'
        settle_df = jan25_df.groupby(['id', 'month_year'], as_index=False)['MAX_area'].sum()
        settle_df.columns = ['id', 'month_year', 'settle_area']
        settle_df['settle_area'] = settle_df['settle_area'].astype(float)

        # Save to a new global variable
        globals()[f'{jan25_var_name}_settle'] = settle_df

In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    long_df = globals()[f'Gemeinde_{y}_100_70_MAX_Jan25']
    settle_df = globals()[f'Gemeinde_{y}_100_70_MAX_Jan25_settle']

    merged_df = pd.merge(long_df, settle_df, on=['id', 'month_year'], how='left')

    globals()[f'Gemeinde_{y}_100_70_MAX_Jan25'] = merged_df


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    df = globals()[f'Gemeinde_{y}_100_70_MAX_Jan25']
    globals()[f'Gemeinde_{y}_100_70_MAX_Jan25'] = df
    df['settle_area_non_zero'] = df.settle_area.replace(0, 1e-6)
    globals()[f'Gemeinde_{y}_100_70_MAX_Jan25']['settle_area_non_zero'] = df.settle_area_non_zero
    globals()[f'Gemeinde_{y}_100_70_MAX_Jan25']['depth_cat_area_pct'] = (df.MAX_area / df.settle_area_non_zero) * 100

In [ ]:
years = [2030, 2050, 2080]

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_70_MAX_Jan25']
    base_df = Gemeinde_2020_100_70_MAX_Jan25
    base_df[f'diff_{year}'] = current_df.MAX_area - base_df.MAX_area

In [ ]:
summary = Gemeinde_2020_100_70_MAX_Jan25[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates().copy()
years = [2030, 2050, 2080]

for year in years:
    diff_col = f'diff_{year}'

    pivot = (
        Gemeinde_2020_100_70_MAX_Jan25
        .groupby(['id', 'depth_cat'])[diff_col]
        .sum()
        .unstack(fill_value=0)
        .add_prefix(f'sum_diff_{year}_')  # e.g., sum_diff_2030_lt0.5
        .reset_index()
    )

    summary = summary.merge(pivot, on='id', how='left')


In [ ]:
# Convert all newly added columns to float
new_cols = [col for col in summary.columns if col.startswith('sum_diff_')]
summary[new_cols] = summary[new_cols].astype(float)

In [ ]:
Gemeinde_2020_100_70_MAX_Jan25[Gemeinde_2020_100_70_MAX_Jan25.Gemeinde == 'Allmendingen']

In [ ]:
summary_70 = summary.copy()

In [ ]:
# Step 1: Calculate total MAX_area per id and depth_cat
area_pivot = (
    Gemeinde_2020_100_70_MAX_Jan25
    .groupby(['id', 'depth_cat'])['MAX_area']
    .sum()
    .unstack(fill_value=0)
    .add_prefix('area_')
    .reset_index()
)

# Step 2: Merge into summary_26
summary_70 = summary_70.merge(area_pivot, on='id', how='left')

# Step 3: Calculate percentage risk levels using area for each category
years = [2030, 2050, 2080]
for year in years:
    # Absolute difference sums (already calculated earlier)
    summary_70[f'high_risk_{year}'] = (
        summary_70.get(f'sum_diff_{year}_lt0.5', 0) +
        summary_70.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_70.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_70[f'very_high_risk_{year}'] = (
        summary_70.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_70.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_70[f'extreme_risk_{year}'] = summary_70.get(f'sum_diff_{year}_gt1.5', 0)

    # Denominators from MAX_area sums
    total_area = (
        summary_70.get('area_lt0.5', 0) +
        summary_70.get('area_lt1.5', 0) +
        summary_70.get('area_gt1.5', 0)
    ).replace(0, 1e-6)

    very_high_area = (
        summary_70.get('area_lt1.5', 0) +
        summary_70.get('area_gt1.5', 0)
    ).replace(0, 1e-6)

    extreme_area = summary_70.get('area_gt1.5', 0).replace(0, 1e-6)

    # Percentage risk levels
    summary_70[f'pct_high_risk_{year}'] = (
        summary_70[f'high_risk_{year}'] / total_area * 100
    ).round(2)

    summary_70[f'pct_very_high_risk_{year}'] = (
        summary_70[f'very_high_risk_{year}'] / very_high_area * 100
    ).round(2)

    summary_70[f'pct_extreme_risk_{year}'] = (
        summary_70[f'extreme_risk_{year}'] / extreme_area * 100
    ).round(2)


In [ ]:
summary_70.columns

In [ ]:
value_vars = [
    'high_risk_2030', 'very_high_risk_2030', 'extreme_risk_2030',
    'high_risk_2050', 'very_high_risk_2050', 'extreme_risk_2050',
    'high_risk_2080', 'very_high_risk_2080', 'extreme_risk_2080'
]

# 1. Melt the absolute values
summary_diff_long = summary_70.melt(
    id_vars=['id', 'Gemeinde', 'Kreis', 'Land'],
    value_vars=value_vars,
    var_name='risk_year',
    value_name='max_diff_2020'
)

# 2. Melt the percent difference columns
summary_pct_long = summary_70.melt(
    id_vars=['id', 'Gemeinde', 'Kreis', 'Land'],
    value_vars=[f'pct_{v}' for v in value_vars],
    var_name='risk_year',
    value_name='max_pct_diff_2020'
)

# 3. Clean `risk_year` for both (they must match for merge)
summary_pct_long['risk_year'] = summary_pct_long['risk_year'].str.replace('pct_', '')

# 4. Merge the two melted DataFrames
summary_70_long = pd.merge(
    summary_diff_long,
    summary_pct_long,
    on=['id', 'Gemeinde', 'Kreis', 'Land', 'risk_year']
)

# 5. Extract risk level and year
summary_70_long[['risk_level', 'year']] = summary_70_long['risk_year'].str.extract(r'(.*)_(\d{4})')
summary_70_long['year'] = summary_70_long['year'].astype(int)


### Include GEM_ID to match with the gemeinden geometries for mapping

In [ ]:
with open("max_summary_26_long.pkl", "rb") as f:
    summary_26_long = pickle.load(f)

In [ ]:
# clean comparison of df1 and df2 by 'id'

# 1. select and sort consistently
df1_clean = summary_70_long[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates()
df2_clean = summary_26_long[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates()

# 2. drop non-Gemeinde entries (e.g., rows where Gemeinde or Land is missing)
df1_clean = df1_clean.dropna(subset=['Gemeinde', 'Land'])
df2_clean = df2_clean.dropna(subset=['Gemeinde', 'Land'])

# 3. set 'id' as index to ensure alignment by identifier, not row order
df1_clean = df1_clean.set_index('id').sort_index()
df2_clean = df2_clean.set_index('id').sort_index()

# 4. align and find differences
# keep only ids present in both datasets
common_ids = df1_clean.index.intersection(df2_clean.index)

# find true mismatches by comparing all columns for each id
diff_mask = ~(df1_clean.loc[common_ids] == df2_clean.loc[common_ids]).all(axis=1)
mismatched_rows = pd.concat(
    [df1_clean.loc[diff_mask], df2_clean.loc[diff_mask]],
    keys=['df1', 'df2']
)

# 5. print results
print(f"number of mismatched ids: {diff_mask.sum()}")
display(mismatched_rows)

# please note that these results are expected and not true mismatches:
# the two "Helgoland" entries in Pinneberg, Schleswig-holstein were renamed with suffixes "-1" and "-2"
# to enable joining with the geometries.
# since there are no settlement areas in these two gemeinden, the results and maps are not affected.

In [ ]:
summary_70_long = summary_70_long.merge(summary_26_long[['id', 'GEM_ID']].drop_duplicates().copy(), on='id', how='left')

### Please store the pre-processed files as pickle files, since you’ll need to load them directly later—especially if the session needs to be restarted due to running out of RAM.

In [ ]:
summary_70_long.to_pickle("max_summary_70_long.pkl")

In [ ]:
with open("max_summary_70_long.pkl", "rb") as f:
    summary_70_long = pickle.load(f)

### Flooding exposure difference map (future years vs. 2020) - prep

In [ ]:
len(summary_70_long)/11112

In [ ]:
for year in [2030, 2050, 2080]:
    globals()[f"summary_70_above_015_{year}"] = summary_70_long[
        (summary_70_long['risk_level'] == 'high_risk') &
        (summary_70_long['Gemeinde'] != 'Germany') &
        (summary_70_long['year'] == year)
    ].copy()


In [ ]:
summary_70_above_015_2030.GEM_ID.nunique()

In [ ]:
admin3_boundary = gpd.read_file('/content/admin-GIS/vg250_01-01.gk3.shape.ebenen/vg250_ebenen_0101/VG250_GEM.shp')

In [ ]:
years = [2030, 2050, 2080]

for y in years:
    summary_70_above_015_df = globals()[f'summary_70_above_015_{y}']

    merged_gdf = admin3_boundary.merge(summary_70_above_015_df, left_on='OBJID', right_on='GEM_ID', how='right')

    globals()[f'summary_70_above_015_{y}_gdf'] = merged_gdf

In [ ]:
years = [2030, 2050, 2080]

for y in years:
    summary_70_above_015_gdf = globals()[f'summary_70_above_015_{y}_gdf']
    print(f"Number of null geometries in summary_70_above_015_{y}_gdf: {len(summary_70_above_015_gdf[summary_70_above_015_gdf.geometry.isna()])}")
    print(f"Number of unique geometries in summary_70_above_015_{y}_gdf: {len(summary_70_above_015_gdf.geometry.unique())}")

In [ ]:
years = [2030, 2050, 2080]
results = {}

for year in years:
    df_name = f"summary_70_above_015_{year}_gdf"
    df = globals()[df_name]   # fetch DataFrame by name

    positive_sum = df.loc[df.max_diff_2020 > 0].max_diff_2020.sum().round(2)
    negative_sum = df.loc[df.max_diff_2020 < 0].max_diff_2020.sum().round(2)

    results[year] = {"positive_sum": positive_sum, "negative_sum": negative_sum}

results


### Percentage difference map

In [ ]:
# Years to plot
years = [2030, 2050, 2080]
save_path = "/content/drive/MyDrive/Germany_Flood_Study/Gemeinde_climate_scenario_maps/final_adjustments"
os.makedirs(save_path, exist_ok=True)

for y in years:
    gdf = globals()[f'summary_70_above_015_{y}_gdf'].copy()
    gdf = gdf.to_crs(epsg=3857)

    # Mask 0 values for transparency
    gdf['masked_pct_diff'] = gdf['max_pct_diff_2020'].replace(0, np.nan)

    # Colors & breaks
    custom_colors = [
        '#40e0d0',  # < -10%
        '#00ffff',  # -10% ~ -1%
        '#808080',  # -1% ~ 1%
        '#fee5d9',  # 1% ~ 10%
        '#fb6a4a',  # 10% ~ 50%
        '#de2d26',  # 50% ~ 100%
        '#a50f15'   # > 100%
    ]
    custom_breaks = [-np.inf, -10, -1, 1, 10, 50, 100, np.inf]

    cmap = ListedColormap(custom_colors)
    norm = BoundaryNorm(custom_breaks, ncolors=len(custom_colors), clip=False)

    # Create figure and axis INSIDE loop
    fig, ax = plt.subplots(figsize=(14, 12))

    # Plot
    gdf.plot(
        ax=ax,
        column='masked_pct_diff',
        cmap=cmap,
        norm=norm,
        edgecolor=None,
        linewidth=0,
        legend=False
    )

    # Overlay geometry boundaries
    gdf.boundary.plot(ax=ax, color='grey', linewidth=0.05)

    # Basemap
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.PositronNoLabels, crs=gdf.crs)

    # Cleanup
    ax.axis('off')

    # North arrow & scale bar
    add_north_arrow(ax, scale=0.75, xlim_pos=0.9025, ylim_pos=0.9,
                    color='black', text_scaler=4, text_yT=-1.25)
    scale1 = ScaleBar(dx=1, location="lower right", scale_loc="bottom")
    ax.add_artist(scale1)

    # Custom legend
    legend_labels = [
        "<-10%", "-10% ~ -1%",
        "-1% ~ 1%",
        "1% ~ 10%", "10% ~ 50%", "50% ~ 100%", ">100%"
    ]
    circles = [
        Line2D([0], [0], marker='o', color='None',
              markerfacecolor=custom_colors[i],
              markeredgecolor='None',
              markersize=10,
              label=legend_labels[i])
        for i in range(len(legend_labels))
    ]
    ax.legend(
        handles=circles,
        title="Exposure change",
        loc='upper left',
        fontsize=12,
        title_fontsize=14
    )

    # Save
    file_name = f"gemeinde_exposure_pct_change_high_risk_70_{y}yr_separate_dry_wet_cyansv2.png"
    full_path = os.path.join(save_path, file_name)
    plt.tight_layout()
    # plt.savefig(full_path, dpi=300, bbox_inches='tight')
    plt.show()
    # plt.close()


## Coastal flooding undefended

### Data Preparation

In [ ]:
# List of years to process
years = [2020, 2030, 2050, 2080]

# Columns you want to select (CU columns and Land)
cu_columns = ['id', 'Gemeinde', 'Kreis', 'Land'] + [col for col in all_columns if col.startswith('CU')]

# Loop through each year and modify the corresponding DataFrame
for year in years:
    if year == 2020:
        df_name = f'Gemeinde_{year}_100_raw'  # 2020 uses '_100_raw'
    else:
        df_name = f'Gemeinde_{year}_100_70_raw'  # 2030, 2050, and 2080 use '_100_85_raw'

    # Check if the DataFrame exists before proceeding
    if df_name in globals():
        # Get the raw DataFrame and select the relevant columns for max flooding data
        df_raw = globals()[df_name]
        globals()[f'Gemeinde_{year}_100_70_CU'] = df_raw[cu_columns]
    else:
        print(f'{df_name} does not exist.')


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    globals()[f'Gemeinde_{y}_100_70_CU_long'] = globals()[f'Gemeinde_{y}_100_70_CU'].melt(
        id_vars=['id', 'Gemeinde', 'Kreis', 'Land'], var_name='month_year_depth', value_name='CU_area'
    )


In [ ]:
Gemeinde_2020_100_70_CU_long.head()

In [ ]:
for df in [Gemeinde_2020_100_70_CU_long, Gemeinde_2030_100_70_CU_long, Gemeinde_2050_100_70_CU_long, Gemeinde_2080_100_70_CU_long]:

    # Replace 'CU' with 'MAX' if needed (optional step if not already 'MAX')
    # df['month_year_depth'] = df['month_year_depth'].str.replace('CU', 'MAX')

    # Split the 'month_year_depth' column into parts
    df[['prefix', 'month_year', 'depth_cat']] = df['month_year_depth'].str.extract(r'^(CU)-([A-Za-z]{3}-\d{2})-(.+)$')

    # Drop 'prefix' if not needed
    df.drop(columns='prefix', inplace=True)


In [ ]:
Gemeinde_2080_100_70_CU_long.head()

In [ ]:
# Define your DataFrames with their corresponding years
year_df_map = {
    2020: Gemeinde_2020_100_70_CU_long,
    2030: Gemeinde_2030_100_70_CU_long,
    2050: Gemeinde_2050_100_70_CU_long,
    2080: Gemeinde_2080_100_70_CU_long
}

# Loop through each DataFrame
for year, df in year_df_map.items():

    # Filter to only Jan-25
    jan25_df = df[df['month_year'] == 'Jan-25'].reset_index(drop=True)

    # Set the new global variable
    globals()[f'Gemeinde_{year}_100_70_CU_Jan25'] = jan25_df


In [ ]:
# Second part: group and create *_settle DataFrames
for year in [2020, 2030, 2050, 2080]:
    jan25_var_name = f'Gemeinde_{year}_100_70_CU_Jan25'

    if jan25_var_name in globals():
        jan25_df = globals()[jan25_var_name]

        # Group and sum by 'id' and 'month_year'
        settle_df = jan25_df.groupby(['id', 'month_year'], as_index=False)['CU_area'].sum()
        settle_df.columns = ['id', 'month_year', 'settle_area']
        settle_df['settle_area'] = settle_df['settle_area'].astype(float)

        # Save to a new global variable
        globals()[f'{jan25_var_name}_settle'] = settle_df

In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    long_df = globals()[f'Gemeinde_{y}_100_70_CU_Jan25']
    settle_df = globals()[f'Gemeinde_{y}_100_70_CU_Jan25_settle']

    merged_df = pd.merge(long_df, settle_df, on=['id', 'month_year'], how='left')

    globals()[f'Gemeinde_{y}_100_70_CU_Jan25'] = merged_df


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    df = globals()[f'Gemeinde_{y}_100_70_CU_Jan25']
    globals()[f'Gemeinde_{y}_100_70_CU_Jan25'] = df
    df['settle_area_non_zero'] = df.settle_area.replace(0, 1e-6)
    globals()[f'Gemeinde_{y}_100_70_CU_Jan25']['settle_area_non_zero'] = df.settle_area_non_zero
    globals()[f'Gemeinde_{y}_100_70_CU_Jan25']['depth_cat_area_pct'] = (df.CU_area / df.settle_area_non_zero) * 100

In [ ]:
cols = ['id', 'Gemeinde', 'Kreis', 'Land']
years = [2030, 2050, 2080]

base_df = Gemeinde_2020_100_70_CU_Jan25

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_70_CU_Jan25']

    # Direct comparison of the column values (row by row, in order)
    is_same = base_df[cols].equals(current_df[cols])

    if is_same:
        print(f"✅ Columns match exactly for year {year}")
    else:
        print(f"❌ Columns differ for year {year}")


In [ ]:
years = [2030, 2050, 2080]

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_70_CU_Jan25']
    base_df = Gemeinde_2020_100_70_CU_Jan25
    base_df[f'diff_{year}'] = current_df.CU_area - base_df.CU_area

In [ ]:
summary = Gemeinde_2020_100_70_CU_Jan25[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates().copy()
years = [2030, 2050, 2080]

for year in years:
    diff_col = f'diff_{year}'

    pivot = (
        Gemeinde_2020_100_70_CU_Jan25
        .groupby(['id', 'depth_cat'])[diff_col]
        .sum()
        .unstack(fill_value=0)
        .add_prefix(f'sum_diff_{year}_')  # e.g., sum_diff_2030_lt0.5
        .reset_index()
    )

    summary = summary.merge(pivot, on='id', how='left')


In [ ]:
# Convert all newly added columns to float
new_cols = [col for col in summary.columns if col.startswith('sum_diff_')]
summary[new_cols] = summary[new_cols].astype(float)

In [ ]:
summary_70 = summary.copy()

In [ ]:
years = [2030, 2050, 2080]

for year in years:
    summary_70[f'high_risk_{year}'] = (
        summary_70.get(f'sum_diff_{year}_lt0.5', 0) +
        summary_70.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_70.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_70[f'very_high_risk_{year}'] = (
        summary_70.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_70.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_70[f'extreme_risk_{year}'] = summary_70.get(f'sum_diff_{year}_gt1.5', 0)

In [ ]:
summary_70.columns

In [ ]:
value_vars = [
    'high_risk_2030', 'very_high_risk_2030', 'extreme_risk_2030',
    'high_risk_2050', 'very_high_risk_2050', 'extreme_risk_2050',
    'high_risk_2080', 'very_high_risk_2080', 'extreme_risk_2080'
]

cu_summary_70_long = summary_70.melt(
    id_vars=['id', 'Gemeinde', 'Kreis', 'Land'],  # keep these
    value_vars=value_vars,
    var_name='risk_year',
    value_name='cu_diff_2020'
)

# Split the combined column into risk_level and year
cu_summary_70_long[['risk_level', 'year']] = cu_summary_70_long['risk_year'].str.extract(r'(.*)_(\d{4})')
cu_summary_70_long['year'] = cu_summary_70_long['year'].astype(int)


### Please store the pre-processed files as pickle files, since you’ll need to load them directly later—especially if the session needs to be restarted due to running out of RAM.

In [ ]:
cu_summary_70_long.to_pickle("cu_summary_70_long.pkl")

In [ ]:
with open('cu_summary_70_long.pkl', 'rb') as f:
    cu_summary_70_long = pickle.load(f)

## Fluvial flooding undefended

### Data Preparation

In [ ]:
# List of years to process
years = [2020, 2030, 2050, 2080]

# Columns you want to select (CU columns and Land)
fu_columns = ['id', 'Gemeinde', 'Kreis', 'Land'] + [col for col in all_columns if col.startswith('FU')]

# Loop through each year and modify the corresponding DataFrame
for year in years:
    if year == 2020:
        df_name = f'Gemeinde_{year}_100_raw'  # 2020 uses '_100_raw'
    else:
        df_name = f'Gemeinde_{year}_100_70_raw'  # 2030, 2050, and 2080 use '_100_85_raw'

    # Check if the DataFrame exists before proceeding
    if df_name in globals():
        # Get the raw DataFrame and select the relevant columns for max flooding data
        df_raw = globals()[df_name]
        globals()[f'Gemeinde_{year}_100_70_FU'] = df_raw[fu_columns]
    else:
        print(f'{df_name} does not exist.')


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    globals()[f'Gemeinde_{y}_100_70_FU_long'] = globals()[f'Gemeinde_{y}_100_70_FU'].melt(
        id_vars=['id', 'Gemeinde', 'Kreis', 'Land'], var_name='month_year_depth', value_name='FU_area'
    )


In [ ]:
Gemeinde_2020_100_70_FU_long.head()

In [ ]:
for df in [Gemeinde_2020_100_70_FU_long, Gemeinde_2030_100_70_FU_long, Gemeinde_2050_100_70_FU_long, Gemeinde_2080_100_70_FU_long]:

    # Replace 'CU' with 'MAX' if needed (optional step if not already 'MAX')
    # df['month_year_depth'] = df['month_year_depth'].str.replace('CU', 'MAX')

    # Split the 'month_year_depth' column into parts
    df[['prefix', 'month_year', 'depth_cat']] = df['month_year_depth'].str.extract(r'^(FU)-([A-Za-z]{3}-\d{2})-(.+)$')

    # Drop 'prefix' if not needed
    df.drop(columns='prefix', inplace=True)


In [ ]:
Gemeinde_2080_100_70_FU_long.head()

In [ ]:
# Define your DataFrames with their corresponding years
year_df_map = {
    2020: Gemeinde_2020_100_70_FU_long,
    2030: Gemeinde_2030_100_70_FU_long,
    2050: Gemeinde_2050_100_70_FU_long,
    2080: Gemeinde_2080_100_70_FU_long
}

# Loop through each DataFrame
for year, df in year_df_map.items():

    # Filter to only Jan-25
    jan25_df = df[df['month_year'] == 'Jan-25'].reset_index(drop=True)

    # Set the new global variable
    globals()[f'Gemeinde_{year}_100_70_FU_Jan25'] = jan25_df


In [ ]:
# Second part: group and create *_settle DataFrames
for year in [2020, 2030, 2050, 2080]:
    jan25_var_name = f'Gemeinde_{year}_100_70_FU_Jan25'

    if jan25_var_name in globals():
        jan25_df = globals()[jan25_var_name]

        # Group and sum by 'id' and 'month_year'
        settle_df = jan25_df.groupby(['id', 'month_year'], as_index=False)['FU_area'].sum()
        settle_df.columns = ['id', 'month_year', 'settle_area']
        settle_df['settle_area'] = settle_df['settle_area'].astype(float)

        # Save to a new global variable
        globals()[f'{jan25_var_name}_settle'] = settle_df

In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    long_df = globals()[f'Gemeinde_{y}_100_70_FU_Jan25']
    settle_df = globals()[f'Gemeinde_{y}_100_70_FU_Jan25_settle']

    merged_df = pd.merge(long_df, settle_df, on=['id', 'month_year'], how='left')

    globals()[f'Gemeinde_{y}_100_70_FU_Jan25'] = merged_df


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    df = globals()[f'Gemeinde_{y}_100_70_FU_Jan25']
    globals()[f'Gemeinde_{y}_100_70_FU_Jan25'] = df
    df['settle_area_non_zero'] = df.settle_area.replace(0, 1e-6)
    globals()[f'Gemeinde_{y}_100_70_FU_Jan25']['settle_area_non_zero'] = df.settle_area_non_zero
    globals()[f'Gemeinde_{y}_100_70_FU_Jan25']['depth_cat_area_pct'] = (df.FU_area / df.settle_area_non_zero) * 100

In [ ]:
cols = ['id', 'Gemeinde', 'Kreis', 'Land']
years = [2030, 2050, 2080]

base_df = Gemeinde_2020_100_70_FU_Jan25

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_70_FU_Jan25']

    # Direct comparison of the column values (row by row, in order)
    is_same = base_df[cols].equals(current_df[cols])

    if is_same:
        print(f"✅ Columns match exactly for year {year}")
    else:
        print(f"❌ Columns differ for year {year}")


In [ ]:
years = [2030, 2050, 2080]

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_70_FU_Jan25']
    base_df = Gemeinde_2020_100_70_FU_Jan25
    base_df[f'diff_{year}'] = current_df.FU_area - base_df.FU_area

In [ ]:
summary = Gemeinde_2020_100_70_FU_Jan25[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates().copy()
years = [2030, 2050, 2080]

for year in years:
    diff_col = f'diff_{year}'

    pivot = (
        Gemeinde_2020_100_70_FU_Jan25
        .groupby(['id', 'depth_cat'])[diff_col]
        .sum()
        .unstack(fill_value=0)
        .add_prefix(f'sum_diff_{year}_')  # e.g., sum_diff_2030_lt0.5
        .reset_index()
    )

    summary = summary.merge(pivot, on='id', how='left')


In [ ]:
# Convert all newly added columns to float
new_cols = [col for col in summary.columns if col.startswith('sum_diff_')]
summary[new_cols] = summary[new_cols].astype(float)

In [ ]:
summary_70 = summary.copy()

In [ ]:
years = [2030, 2050, 2080]

for year in years:
    summary_70[f'high_risk_{year}'] = (
        summary_70.get(f'sum_diff_{year}_lt0.5', 0) +
        summary_70.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_70.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_70[f'very_high_risk_{year}'] = (
        summary_70.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_70.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_70[f'extreme_risk_{year}'] = summary_70.get(f'sum_diff_{year}_gt1.5', 0)

In [ ]:
summary_70.columns

In [ ]:
value_vars = [
    'high_risk_2030', 'very_high_risk_2030', 'extreme_risk_2030',
    'high_risk_2050', 'very_high_risk_2050', 'extreme_risk_2050',
    'high_risk_2080', 'very_high_risk_2080', 'extreme_risk_2080'
]

fu_summary_70_long = summary_70.melt(
    id_vars=['id', 'Gemeinde', 'Kreis', 'Land'],  # keep these
    value_vars=value_vars,
    var_name='risk_year',
    value_name='fu_diff_2020'
)

# Split the combined column into risk_level and year
fu_summary_70_long[['risk_level', 'year']] = fu_summary_70_long['risk_year'].str.extract(r'(.*)_(\d{4})')
fu_summary_70_long['year'] = fu_summary_70_long['year'].astype(int)


### Please store the pre-processed files as pickle files, since you’ll need to load them directly later—especially if the session needs to be restarted due to running out of RAM.

In [ ]:
fu_summary_70_long.to_pickle("fu_summary_70_long.pkl")

In [ ]:
with open('fu_summary_70_long.pkl', 'rb') as f:
    fu_summary_70_long = pickle.load(f)

## Pluvial flooding defended

### Data Preparation

In [ ]:
# List of years to process
years = [2020, 2030, 2050, 2080]

# Columns you want to select (CU columns and Land)
pd_columns = ['id', 'Gemeinde', 'Kreis', 'Land'] + [col for col in all_columns if col.startswith('PD') or col == 'Land']

# Loop through each year and modify the corresponding DataFrame
for year in years:
    if year == 2020:
        df_name = f'Gemeinde_{year}_100_raw'  # 2020 uses '_100_raw'
    else:
        df_name = f'Gemeinde_{year}_100_70_raw'  # 2030, 2050, and 2080 use '_100_85_raw'

    # Check if the DataFrame exists before proceeding
    if df_name in globals():
        # Get the raw DataFrame and select the relevant columns for max flooding data
        df_raw = globals()[df_name]
        globals()[f'Gemeinde_{year}_100_70_PD'] = df_raw[pd_columns]
    else:
        print(f'{df_name} does not exist.')


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    globals()[f'Gemeinde_{y}_100_70_PD_long'] = globals()[f'Gemeinde_{y}_100_70_PD'].melt(
        id_vars=['id', 'Gemeinde', 'Kreis', 'Land'], var_name='month_year_depth', value_name='PD_area'
    )


In [ ]:
Gemeinde_2020_100_70_PD_long.head()

In [ ]:
for df in [Gemeinde_2020_100_70_PD_long, Gemeinde_2030_100_70_PD_long, Gemeinde_2050_100_70_PD_long, Gemeinde_2080_100_70_PD_long]:

    # Replace 'CU' with 'MAX' if needed (optional step if not already 'MAX')
    # df['month_year_depth'] = df['month_year_depth'].str.replace('CU', 'MAX')

    # Split the 'month_year_depth' column into parts
    df[['prefix', 'month_year', 'depth_cat']] = df['month_year_depth'].str.extract(r'^(PD)-([A-Za-z]{3}-\d{2})-(.+)$')

    # Drop 'prefix' if not needed
    df.drop(columns='prefix', inplace=True)


In [ ]:
Gemeinde_2080_100_70_PD_long.head()

In [ ]:
# Define your DataFrames with their corresponding years
year_df_map = {
    2020: Gemeinde_2020_100_70_PD_long,
    2030: Gemeinde_2030_100_70_PD_long,
    2050: Gemeinde_2050_100_70_PD_long,
    2080: Gemeinde_2080_100_70_PD_long
}

# Loop through each DataFrame
for year, df in year_df_map.items():

    # Filter to only Jan-25
    jan25_df = df[df['month_year'] == 'Jan-25'].reset_index(drop=True)

    # Set the new global variable
    globals()[f'Gemeinde_{year}_100_70_PD_Jan25'] = jan25_df


In [ ]:
# Second part: group and create *_settle DataFrames
for year in [2020, 2030, 2050, 2080]:
    jan25_var_name = f'Gemeinde_{year}_100_70_PD_Jan25'

    if jan25_var_name in globals():
        jan25_df = globals()[jan25_var_name]

        # Group and sum by 'id' and 'month_year'
        settle_df = jan25_df.groupby(['id', 'month_year'], as_index=False)['PD_area'].sum()
        settle_df.columns = ['id', 'month_year', 'settle_area']
        settle_df['settle_area'] = settle_df['settle_area'].astype(float)

        # Save to a new global variable
        globals()[f'{jan25_var_name}_settle'] = settle_df

In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    long_df = globals()[f'Gemeinde_{y}_100_70_PD_Jan25']
    settle_df = globals()[f'Gemeinde_{y}_100_70_PD_Jan25_settle']

    merged_df = pd.merge(long_df, settle_df, on=['id', 'month_year'], how='left')

    globals()[f'Gemeinde_{y}_100_70_PD_Jan25'] = merged_df


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    df = globals()[f'Gemeinde_{y}_100_70_PD_Jan25']
    globals()[f'Gemeinde_{y}_100_70_PD_Jan25'] = df
    df['settle_area_non_zero'] = df.settle_area.replace(0, 1e-6)
    globals()[f'Gemeinde_{y}_100_70_PD_Jan25']['settle_area_non_zero'] = df.settle_area_non_zero
    globals()[f'Gemeinde_{y}_100_70_PD_Jan25']['depth_cat_area_pct'] = (df.PD_area / df.settle_area_non_zero) * 100

In [ ]:
cols = ['id', 'Gemeinde', 'Kreis', 'Land']
years = [2030, 2050, 2080]

base_df = Gemeinde_2020_100_70_PD_Jan25

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_70_PD_Jan25']

    # Direct comparison of the column values (row by row, in order)
    is_same = base_df[cols].equals(current_df[cols])

    if is_same:
        print(f"✅ Columns match exactly for year {year}")
    else:
        print(f"❌ Columns differ for year {year}")


In [ ]:
years = [2030, 2050, 2080]

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_70_PD_Jan25']
    base_df = Gemeinde_2020_100_70_PD_Jan25
    base_df[f'diff_{year}'] = current_df.PD_area - base_df.PD_area

In [ ]:
summary = Gemeinde_2020_100_70_PD_Jan25[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates().copy()
years = [2030, 2050, 2080]

for year in years:
    diff_col = f'diff_{year}'

    pivot = (
        Gemeinde_2020_100_70_PD_Jan25
        .groupby(['id', 'depth_cat'])[diff_col]
        .sum()
        .unstack(fill_value=0)
        .add_prefix(f'sum_diff_{year}_')  # e.g., sum_diff_2030_lt0.5
        .reset_index()
    )

    summary = summary.merge(pivot, on='id', how='left')


In [ ]:
# Convert all newly added columns to float
new_cols = [col for col in summary.columns if col.startswith('sum_diff_')]
summary[new_cols] = summary[new_cols].astype(float)

In [ ]:
summary_70 = summary.copy()

In [ ]:
years = [2030, 2050, 2080]

for year in years:
    summary_70[f'high_risk_{year}'] = (
        summary_70.get(f'sum_diff_{year}_lt0.5', 0) +
        summary_70.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_70.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_70[f'very_high_risk_{year}'] = (
        summary_70.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_70.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_70[f'extreme_risk_{year}'] = summary_70.get(f'sum_diff_{year}_gt1.5', 0)

In [ ]:
summary_70.columns

In [ ]:
value_vars = [
    'high_risk_2030', 'very_high_risk_2030', 'extreme_risk_2030',
    'high_risk_2050', 'very_high_risk_2050', 'extreme_risk_2050',
    'high_risk_2080', 'very_high_risk_2080', 'extreme_risk_2080'
]

pd_summary_70_long = summary_70.melt(
    id_vars=['id', 'Gemeinde', 'Kreis', 'Land'],  # keep these
    value_vars=value_vars,
    var_name='risk_year',
    value_name='pd_diff_2020'
)

# Split the combined column into risk_level and year
pd_summary_70_long[['risk_level', 'year']] = pd_summary_70_long['risk_year'].str.extract(r'(.*)_(\d{4})')
pd_summary_70_long['year'] = pd_summary_70_long['year'].astype(int)


### Please store the pre-processed files as pickle files, since you’ll need to load them directly later—especially if the session needs to be restarted due to running out of RAM.

In [ ]:
pd_summary_70_long.to_pickle("pd_summary_70_long.pkl")

In [ ]:
with open("pd_summary_70_long.pkl", "rb") as f:
    pd_summary_70_long = pickle.load(f)

# SSP5-RCP8.5 (Most Pessimistic)

### Read files, pre-processing and store

In [ ]:
!unzip "/content/drive/MyDrive/Germany_Flood_Study/Germany_flood_study_raw_csv/most_pessimistic.zip"

In [ ]:
file_paths = [
    '/content/Germany_Gemeinde_2020_100.xlsx',
    '/content/most_pessimistic/Germany_Gemeinde_2030_100_85.xlsx',
    '/content/most_pessimistic/Germany_Gemeinde_2050_100_85.xlsx',
    '/content/most_pessimistic/Germany_Gemeinde_2080_100_85.xlsx',
]

dataframes = []
for path in file_paths:
    df = pd.read_excel(path)
    df = df.round(6)
    dataframes.append(df)

# Optionally unpack into variables
(Gemeinde_2020_100_raw, Gemeinde_2030_100_85_raw, Gemeinde_2050_100_85_raw, Gemeinde_2080_100_85_raw) = dataframes

In [ ]:
Gemeinde_2020_100_raw.head()

In [ ]:
# Date range: every 6 months
dates = pd.date_range(start='2016-07-01', end='2025-01-01', freq='6MS')

# Prefixes and suffixes
prefixes = ['CU', 'FU', 'PD', 'MAX']
suffixes = ['P0', 'lt0.15', 'lt0.5', 'lt1.5', 'gt1.5']

# Correct order: by prefix, then suffix, then date
all_columns = [
    f"{prefix}-{date.strftime('%b-%y')}-{suffix}"
    for prefix in prefixes
    for suffix in suffixes
    for date in dates
]

In [ ]:
# Check if number of replacement columns matches the shape
for df in [Gemeinde_2020_100_raw, Gemeinde_2030_100_85_raw, Gemeinde_2050_100_85_raw, Gemeinde_2080_100_85_raw]:
    if len(all_columns) == df.shape[1] - 3:
      df.columns = list(df.columns[:3]) + all_columns
      df.reset_index(drop=True, inplace=True)
      df['id'] = df.index
    else:
      raise ValueError("Length of generated column names does not match number of columns (excluding the first one).")


In [ ]:
Gemeinde_2030_100_85_raw.head()

In [ ]:
Gemeinde_2030_100_85_raw[1:].head()

In [ ]:
Gemeinde_2020_100_raw = Gemeinde_2020_100_raw[1:]
Gemeinde_2030_100_85_raw = Gemeinde_2030_100_85_raw[1:]
Gemeinde_2050_100_85_raw = Gemeinde_2050_100_85_raw[1:]
Gemeinde_2080_100_85_raw = Gemeinde_2080_100_85_raw[1:]

In [ ]:
save_path = "/content/"

# Make sure the directory exists

os.makedirs(save_path, exist_ok=True)

gemeinde_dfs = [Gemeinde_2020_100_raw, Gemeinde_2030_100_85_raw, Gemeinde_2050_100_85_raw, Gemeinde_2080_100_85_raw]

filenames = [
    "Gemeinde_2020_100_raw.pkl", "Gemeinde_2030_100_85_aw.pkl", "Gemeinde_2050_100_85_raw.pkl", "Gemeinde_2080_100_85_raw.pkl"
]

for df, filename in zip(gemeinde_dfs, filenames):
    df.to_pickle(os.path.join(save_path, filename))


### **PLEASE re-run if the session restarts before you finish processing this climate scenario:** Read and load file from here

In [ ]:
# Define the path where the files are stored
save_path = "/content/"

# List of filenames
filenames = [
    "Gemeinde_2020_100_raw.pkl", "Gemeinde_2030_100_85_aw.pkl", "Gemeinde_2050_100_85_raw.pkl", "Gemeinde_2080_100_85_raw.pkl"
]

# Load all DataFrames into a list
gemeinde_dfs = [pd.read_pickle(os.path.join(save_path, filename)) for filename in filenames]

cols_to_convert = [col for col in gemeinde_dfs[0].columns if col not in ['Gemeinde', 'Kreis', 'Land', 'id']]

def convert_and_round(df):
    df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce').round(6)
    return df

gemeinde_dfs = [convert_and_round(df) for df in gemeinde_dfs]

# Optionally assign them back to individual variable names (if needed)
(
  Gemeinde_2020_100_raw,
  Gemeinde_2030_100_85_raw,
  Gemeinde_2050_100_85_raw,
  Gemeinde_2080_100_85_raw
) = gemeinde_dfs


In [ ]:
# Date range: every 6 months
dates = pd.date_range(start='2016-07-01', end='2025-01-01', freq='6MS')

# Prefixes and suffixes
prefixes = ['CU', 'FU', 'PD', 'MAX']
suffixes = ['P0', 'lt0.15', 'lt0.5', 'lt1.5', 'gt1.5']

# Correct order: by prefix, then suffix, then date
all_columns = [
    f"{prefix}-{date.strftime('%b-%y')}-{suffix}"
    for prefix in prefixes
    for suffix in suffixes
    for date in dates
]

## Combined flooding (max)

### Data preparation

In [ ]:
# List of years to process
years = [2020, 2030, 2050, 2080]

# Columns you want to select (CU columns and Land)
max_columns = ['id', 'Gemeinde', 'Kreis', 'Land'] + [col for col in all_columns if col.startswith('MAX') or col == 'Land']

# Loop through each year and modify the corresponding DataFrame
for year in years:
    if year == 2020:
        df_name = f'Gemeinde_{year}_100_raw'  # 2020 uses '_100_raw'
    else:
        df_name = f'Gemeinde_{year}_100_85_raw'  # 2030, 2050, and 2080 use '_100_85_raw'

    # Check if the DataFrame exists before proceeding
    if df_name in globals():
        # Get the raw DataFrame and select the relevant columns for max flooding data
        df_raw = globals()[df_name]
        globals()[f'Gemeinde_{year}_100_85_MAX'] = df_raw[max_columns]
    else:
        print(f'{df_name} does not exist.')


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    globals()[f'Gemeinde_{y}_100_85_MAX_long'] = globals()[f'Gemeinde_{y}_100_85_MAX'].melt(
        id_vars=['id', 'Gemeinde', 'Kreis', 'Land'], var_name='month_year_depth', value_name='MAX_area'
    )


In [ ]:
Gemeinde_2020_100_85_MAX_long.head()

In [ ]:
for df in [Gemeinde_2020_100_85_MAX_long, Gemeinde_2030_100_85_MAX_long, Gemeinde_2050_100_85_MAX_long, Gemeinde_2080_100_85_MAX_long]:

    # Replace 'CU' with 'MAX' if needed (optional step if not already 'MAX')
    # df['month_year_depth'] = df['month_year_depth'].str.replace('CU', 'MAX')

    # Split the 'month_year_depth' column into parts
    df[['prefix', 'month_year', 'depth_cat']] = df['month_year_depth'].str.extract(r'^(MAX)-([A-Za-z]{3}-\d{2})-(.+)$')

    # Drop 'prefix' if not needed
    df.drop(columns='prefix', inplace=True)


In [ ]:
Gemeinde_2080_100_85_MAX_long.head()

In [ ]:
# Define your DataFrames with their corresponding years
year_df_map = {
    2020: Gemeinde_2020_100_85_MAX_long,
    2030: Gemeinde_2030_100_85_MAX_long,
    2050: Gemeinde_2050_100_85_MAX_long,
    2080: Gemeinde_2080_100_85_MAX_long
}

# Loop through each DataFrame
for year, df in year_df_map.items():

    # Filter to only Jan-25
    jan25_df = df[df['month_year'] == 'Jan-25'].reset_index(drop=True)

    # Set the new global variable
    globals()[f'Gemeinde_{year}_100_85_MAX_Jan25'] = jan25_df


In [ ]:
# Second part: group and create *_settle DataFrames
for year in [2020, 2030, 2050, 2080]:
    jan25_var_name = f'Gemeinde_{year}_100_85_MAX_Jan25'

    if jan25_var_name in globals():
        jan25_df = globals()[jan25_var_name]

        # Group and sum by 'id' and 'month_year'
        settle_df = jan25_df.groupby(['id', 'month_year'], as_index=False)['MAX_area'].sum()
        settle_df.columns = ['id', 'month_year', 'settle_area']
        settle_df['settle_area'] = settle_df['settle_area'].astype(float)

        # Save to a new global variable
        globals()[f'{jan25_var_name}_settle'] = settle_df

In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    long_df = globals()[f'Gemeinde_{y}_100_85_MAX_Jan25']
    settle_df = globals()[f'Gemeinde_{y}_100_85_MAX_Jan25_settle']

    merged_df = pd.merge(long_df, settle_df, on=['id', 'month_year'], how='left')

    globals()[f'Gemeinde_{y}_100_85_MAX_Jan25'] = merged_df


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    df = globals()[f'Gemeinde_{y}_100_85_MAX_Jan25']
    globals()[f'Gemeinde_{y}_100_85_MAX_Jan25'] = df
    df['settle_area_non_zero'] = df.settle_area.replace(0, 1e-6)
    globals()[f'Gemeinde_{y}_100_85_MAX_Jan25']['settle_area_non_zero'] = df.settle_area_non_zero
    globals()[f'Gemeinde_{y}_100_85_MAX_Jan25']['depth_cat_area_pct'] = (df.MAX_area / df.settle_area_non_zero) * 100

In [ ]:
years = [2030, 2050, 2080]

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_85_MAX_Jan25']
    base_df = Gemeinde_2020_100_85_MAX_Jan25
    base_df[f'diff_{year}'] = current_df.MAX_area - base_df.MAX_area

In [ ]:
summary = Gemeinde_2020_100_85_MAX_Jan25[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates().copy()
years = [2030, 2050, 2080]

for year in years:
    diff_col = f'diff_{year}'

    pivot = (
        Gemeinde_2020_100_85_MAX_Jan25
        .groupby(['id', 'depth_cat'])[diff_col]
        .sum()
        .unstack(fill_value=0)
        .add_prefix(f'sum_diff_{year}_')  # e.g., sum_diff_2030_lt0.5
        .reset_index()
    )

    summary = summary.merge(pivot, on='id', how='left')


In [ ]:
# Convert all newly added columns to float
new_cols = [col for col in summary.columns if col.startswith('sum_diff_')]
summary[new_cols] = summary[new_cols].astype(float)

In [ ]:
summary_85 = summary.copy()

In [ ]:
# Step 1: Calculate total MAX_area per id and depth_cat
area_pivot = (
    Gemeinde_2020_100_85_MAX_Jan25
    .groupby(['id', 'depth_cat'])['MAX_area']
    .sum()
    .unstack(fill_value=0)
    .add_prefix('area_')
    .reset_index()
)

# Step 2: Merge into summary_26
summary_85 = summary_85.merge(area_pivot, on='id', how='left')

# Step 3: Calculate percentage risk levels using area for each category
years = [2030, 2050, 2080]
for year in years:
    # Absolute difference sums (already calculated earlier)
    summary_85[f'high_risk_{year}'] = (
        summary_85.get(f'sum_diff_{year}_lt0.5', 0) +
        summary_85.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_85.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_85[f'very_high_risk_{year}'] = (
        summary_85.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_85.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_85[f'extreme_risk_{year}'] = summary_85.get(f'sum_diff_{year}_gt1.5', 0)

    # Denominators from MAX_area sums
    total_area = (
        summary_85.get('area_lt0.5', 0) +
        summary_85.get('area_lt1.5', 0) +
        summary_85.get('area_gt1.5', 0)
    ).replace(0, 1e-6)

    very_high_area = (
        summary_85.get('area_lt1.5', 0) +
        summary_85.get('area_gt1.5', 0)
    ).replace(0, 1e-6)

    extreme_area = summary_85.get('area_gt1.5', 0).replace(0, 1e-6)

    # Percentage risk levels
    summary_85[f'pct_high_risk_{year}'] = (
        summary_85[f'high_risk_{year}'] / total_area * 100
    ).round(2)

    summary_85[f'pct_very_high_risk_{year}'] = (
        summary_85[f'very_high_risk_{year}'] / very_high_area * 100
    ).round(2)

    summary_85[f'pct_extreme_risk_{year}'] = (
        summary_85[f'extreme_risk_{year}'] / extreme_area * 100
    ).round(2)


In [ ]:
summary_85.columns

In [ ]:
value_vars = [
    'high_risk_2030', 'very_high_risk_2030', 'extreme_risk_2030',
    'high_risk_2050', 'very_high_risk_2050', 'extreme_risk_2050',
    'high_risk_2080', 'very_high_risk_2080', 'extreme_risk_2080'
]

# 1. Melt the absolute values
summary_diff_long = summary_85.melt(
    id_vars=['id', 'Gemeinde', 'Kreis', 'Land'],
    value_vars=value_vars,
    var_name='risk_year',
    value_name='max_diff_2020'
)

# 2. Melt the percent difference columns
summary_pct_long = summary_85.melt(
    id_vars=['id', 'Gemeinde', 'Kreis', 'Land'],
    value_vars=[f'pct_{v}' for v in value_vars],
    var_name='risk_year',
    value_name='max_pct_diff_2020'
)

# 3. Clean `risk_year` for both (they must match for merge)
summary_pct_long['risk_year'] = summary_pct_long['risk_year'].str.replace('pct_', '')

# 4. Merge the two melted DataFrames
summary_85_long = pd.merge(
    summary_diff_long,
    summary_pct_long,
    on=['id', 'Gemeinde', 'Kreis', 'Land', 'risk_year']
)

# 5. Extract risk level and year
summary_85_long[['risk_level', 'year']] = summary_85_long['risk_year'].str.extract(r'(.*)_(\d{4})')
summary_85_long['year'] = summary_85_long['year'].astype(int)


### Include GEM_ID to match with the gemeinden geometries for mapping

In [ ]:
with open("max_summary_26_long.pkl", "rb") as f:
    summary_26_long = pickle.load(f)

In [ ]:
# clean comparison of df1 and df2 by 'id'

# 1. select and sort consistently
df1_clean = summary_85_long[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates()
df2_clean = summary_26_long[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates()

# 2. drop non-Gemeinde entries (e.g., rows where Gemeinde or Land is missing)
df1_clean = df1_clean.dropna(subset=['Gemeinde', 'Land'])
df2_clean = df2_clean.dropna(subset=['Gemeinde', 'Land'])

# 3. set 'id' as index to ensure alignment by identifier, not row order
df1_clean = df1_clean.set_index('id').sort_index()
df2_clean = df2_clean.set_index('id').sort_index()

# 4. align and find differences
# keep only ids present in both datasets
common_ids = df1_clean.index.intersection(df2_clean.index)

# find true mismatches by comparing all columns for each id
diff_mask = ~(df1_clean.loc[common_ids] == df2_clean.loc[common_ids]).all(axis=1)
mismatched_rows = pd.concat(
    [df1_clean.loc[diff_mask], df2_clean.loc[diff_mask]],
    keys=['df1', 'df2']
)

# 5. print results
print(f"number of mismatched ids: {diff_mask.sum()}")
display(mismatched_rows)

# please note that these results are expected and not true mismatches:
# the two "Helgoland" entries in Pinneberg, Schleswig-holstein were renamed with suffixes "-1" and "-2"
# to enable joining with the geometries.
# since there are no settlement areas in these two gemeinden, the results and maps are not affected.

In [ ]:
summary_85_long = summary_85_long.merge(summary_26_long[['id', 'GEM_ID']].drop_duplicates().copy(), on='id', how='left')

### Please store the pre-processed files as pickle files, since you’ll need to load them directly later—especially if the session needs to be restarted due to running out of RAM.

In [ ]:
summary_85_long.to_pickle("max_summary_85_long.pkl")

In [ ]:
with open("max_summary_85_long.pkl", "rb") as f:
    summary_85_long = pickle.load(f)

### Flooding exposure difference map (future years vs. 2020) - prep

In [ ]:
for year in [2030, 2050, 2080]:
    globals()[f"summary_85_above_015_{year}"] = summary_85_long[
        (summary_85_long['risk_level'] == 'high_risk') &
        (summary_85_long['Gemeinde'] != 'Germany') &
        (summary_85_long['year'] == year)
    ].copy()


In [ ]:
summary_85_above_015_2030.GEM_ID.nunique()

In [ ]:
admin3_boundary = gpd.read_file('/content/admin-GIS/vg250_01-01.gk3.shape.ebenen/vg250_ebenen_0101/VG250_GEM.shp')

In [ ]:
years = [2030, 2050, 2080]

for y in years:
    summary_85_above_015_df = globals()[f'summary_85_above_015_{y}']

    merged_gdf = admin3_boundary.merge(summary_85_above_015_df, left_on='OBJID', right_on='GEM_ID', how='right')

    globals()[f'summary_85_above_015_{y}_gdf'] = merged_gdf

In [ ]:
years = [2030, 2050, 2080]

for y in years:
    summary_85_above_015_gdf = globals()[f'summary_85_above_015_{y}_gdf']
    print(f"Number of null geometries in summary_85_above_015_{y}_gdf: {len(summary_85_above_015_gdf[summary_85_above_015_gdf.geometry.isna()])}")
    print(f"Number of unique geometries in summary_85_above_015_{y}_gdf: {len(summary_85_above_015_gdf.geometry.unique())}")

In [ ]:
years = [2030, 2050, 2080]
results = {}

for year in years:
    df_name = f"summary_85_above_015_{year}_gdf"
    df = globals()[df_name]   # fetch DataFrame by name

    positive_sum = df.loc[df.max_diff_2020 > 0].max_diff_2020.sum().round(2)
    negative_sum = df.loc[df.max_diff_2020 < 0].max_diff_2020.sum().round(2)

    results[year] = {"positive_sum": positive_sum, "negative_sum": negative_sum}

results


### Percentage difference map

In [ ]:
# Years to plot
years = [2030, 2050, 2080]
save_path = "/content/drive/MyDrive/Germany_Flood_Study/Gemeinde_climate_scenario_maps/final_adjustments"
os.makedirs(save_path, exist_ok=True)

for y in years:
    gdf = globals()[f'summary_85_above_015_{y}_gdf'].copy()
    gdf = gdf.to_crs(epsg=3857)

    # Mask 0 values for transparency
    gdf['masked_pct_diff'] = gdf['max_pct_diff_2020'].replace(0, np.nan)

    # Colors & breaks
    custom_colors = [
        '#40e0d0',  # < -10%
        '#00ffff',  # -10% ~ -1%
        '#808080',  # -1% ~ 1%
        '#fee5d9',  # 1% ~ 10%
        '#fb6a4a',  # 10% ~ 50%
        '#de2d26',  # 50% ~ 100%
        '#a50f15'   # > 100%
    ]
    custom_breaks = [-np.inf, -10, -1, 1, 10, 50, 100, np.inf]

    cmap = ListedColormap(custom_colors)
    norm = BoundaryNorm(custom_breaks, ncolors=len(custom_colors), clip=False)

    # Create figure and axis INSIDE loop
    fig, ax = plt.subplots(figsize=(14, 12))

    # Plot
    gdf.plot(
        ax=ax,
        column='masked_pct_diff',
        cmap=cmap,
        norm=norm,
        edgecolor=None,
        linewidth=0,
        legend=False
    )

    # Overlay geometry boundaries
    gdf.boundary.plot(ax=ax, color='grey', linewidth=0.05)

    # Basemap
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.PositronNoLabels, crs=gdf.crs)

    # Cleanup
    ax.axis('off')

    # North arrow & scale bar
    add_north_arrow(ax, scale=0.75, xlim_pos=0.9025, ylim_pos=0.9,
                    color='black', text_scaler=4, text_yT=-1.25)
    scale1 = ScaleBar(dx=1, location="lower right", scale_loc="bottom")
    ax.add_artist(scale1)

    # Custom legend
    legend_labels = [
        "<-10%", "-10% ~ -1%",
        "-1% ~ 1%",
        "1% ~ 10%", "10% ~ 50%", "50% ~ 100%", ">100%"
    ]
    circles = [
        Line2D([0], [0], marker='o', color='None',
              markerfacecolor=custom_colors[i],
              markeredgecolor='None',
              markersize=10,
              label=legend_labels[i])
        for i in range(len(legend_labels))
    ]
    ax.legend(
        handles=circles,
        title="Exposure change",
        loc='upper left',
        fontsize=12,
        title_fontsize=14
    )

    # Save
    file_name = f"gemeinde_exposure_pct_change_high_risk_85_{y}yr_separate_dry_wet_cyansv2.png"
    full_path = os.path.join(save_path, file_name)
    plt.tight_layout()
    # plt.savefig(full_path, dpi=300, bbox_inches='tight')
    plt.show()
    # plt.close()


## Coastal flooding undefended

### Data Preparation

In [ ]:
# List of years to process
years = [2020, 2030, 2050, 2080]

# Columns you want to select (CU columns and Land)
cu_columns = ['id', 'Gemeinde', 'Kreis', 'Land'] + [col for col in all_columns if col.startswith('CU')]

# Loop through each year and modify the corresponding DataFrame
for year in years:
    if year == 2020:
        df_name = f'Gemeinde_{year}_100_raw'  # 2020 uses '_100_raw'
    else:
        df_name = f'Gemeinde_{year}_100_85_raw'  # 2030, 2050, and 2080 use '_100_85_raw'

    # Check if the DataFrame exists before proceeding
    if df_name in globals():
        # Get the raw DataFrame and select the relevant columns for max flooding data
        df_raw = globals()[df_name]
        globals()[f'Gemeinde_{year}_100_85_CU'] = df_raw[cu_columns]
    else:
        print(f'{df_name} does not exist.')


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    globals()[f'Gemeinde_{y}_100_85_CU_long'] = globals()[f'Gemeinde_{y}_100_85_CU'].melt(
        id_vars=['id', 'Gemeinde', 'Kreis', 'Land'], var_name='month_year_depth', value_name='CU_area'
    )


In [ ]:
Gemeinde_2020_100_85_CU_long.head()

In [ ]:
for df in [Gemeinde_2020_100_85_CU_long, Gemeinde_2030_100_85_CU_long, Gemeinde_2050_100_85_CU_long, Gemeinde_2080_100_85_CU_long]:

    # Replace 'CU' with 'MAX' if needed (optional step if not already 'MAX')
    # df['month_year_depth'] = df['month_year_depth'].str.replace('CU', 'MAX')

    # Split the 'month_year_depth' column into parts
    df[['prefix', 'month_year', 'depth_cat']] = df['month_year_depth'].str.extract(r'^(CU)-([A-Za-z]{3}-\d{2})-(.+)$')

    # Drop 'prefix' if not needed
    df.drop(columns='prefix', inplace=True)


In [ ]:
Gemeinde_2080_100_85_CU_long.head()

In [ ]:
# Define your DataFrames with their corresponding years
year_df_map = {
    2020: Gemeinde_2020_100_85_CU_long,
    2030: Gemeinde_2030_100_85_CU_long,
    2050: Gemeinde_2050_100_85_CU_long,
    2080: Gemeinde_2080_100_85_CU_long
}

# Loop through each DataFrame
for year, df in year_df_map.items():

    # Filter to only Jan-25
    jan25_df = df[df['month_year'] == 'Jan-25'].reset_index(drop=True)

    # Set the new global variable
    globals()[f'Gemeinde_{year}_100_85_CU_Jan25'] = jan25_df


In [ ]:
# Second part: group and create *_settle DataFrames
for year in [2020, 2030, 2050, 2080]:
    jan25_var_name = f'Gemeinde_{year}_100_85_CU_Jan25'

    if jan25_var_name in globals():
        jan25_df = globals()[jan25_var_name]

        # Group and sum by 'id' and 'month_year'
        settle_df = jan25_df.groupby(['id', 'month_year'], as_index=False)['CU_area'].sum()
        settle_df.columns = ['id', 'month_year', 'settle_area']
        settle_df['settle_area'] = settle_df['settle_area'].astype(float)

        # Save to a new global variable
        globals()[f'{jan25_var_name}_settle'] = settle_df

In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    long_df = globals()[f'Gemeinde_{y}_100_85_CU_Jan25']
    settle_df = globals()[f'Gemeinde_{y}_100_85_CU_Jan25_settle']

    merged_df = pd.merge(long_df, settle_df, on=['id', 'month_year'], how='left')

    globals()[f'Gemeinde_{y}_100_85_CU_Jan25'] = merged_df


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    df = globals()[f'Gemeinde_{y}_100_85_CU_Jan25']
    globals()[f'Gemeinde_{y}_100_85_CU_Jan25'] = df
    df['settle_area_non_zero'] = df.settle_area.replace(0, 1e-6)
    globals()[f'Gemeinde_{y}_100_85_CU_Jan25']['settle_area_non_zero'] = df.settle_area_non_zero
    globals()[f'Gemeinde_{y}_100_85_CU_Jan25']['depth_cat_area_pct'] = (df.CU_area / df.settle_area_non_zero) * 100

In [ ]:
cols = ['id', 'Gemeinde', 'Kreis', 'Land']
years = [2030, 2050, 2080]

base_df = Gemeinde_2020_100_85_CU_Jan25

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_85_CU_Jan25']

    # Direct comparison of the column values (row by row, in order)
    is_same = base_df[cols].equals(current_df[cols])

    if is_same:
        print(f"✅ Columns match exactly for year {year}")
    else:
        print(f"❌ Columns differ for year {year}")


In [ ]:
years = [2030, 2050, 2080]

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_85_CU_Jan25']
    base_df = Gemeinde_2020_100_85_CU_Jan25
    base_df[f'diff_{year}'] = current_df.CU_area - base_df.CU_area

In [ ]:
summary = Gemeinde_2020_100_85_CU_Jan25[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates().copy()
years = [2030, 2050, 2080]

for year in years:
    diff_col = f'diff_{year}'

    pivot = (
        Gemeinde_2020_100_85_CU_Jan25
        .groupby(['id', 'depth_cat'])[diff_col]
        .sum()
        .unstack(fill_value=0)
        .add_prefix(f'sum_diff_{year}_')  # e.g., sum_diff_2030_lt0.5
        .reset_index()
    )

    summary = summary.merge(pivot, on='id', how='left')


In [ ]:
# Convert all newly added columns to float
new_cols = [col for col in summary.columns if col.startswith('sum_diff_')]
summary[new_cols] = summary[new_cols].astype(float)

In [ ]:
summary_85 = summary.copy()

In [ ]:
years = [2030, 2050, 2080]

for year in years:
    summary_85[f'high_risk_{year}'] = (
        summary_85.get(f'sum_diff_{year}_lt0.5', 0) +
        summary_85.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_85.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_85[f'very_high_risk_{year}'] = (
        summary_85.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_85.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_85[f'extreme_risk_{year}'] = summary_85.get(f'sum_diff_{year}_gt1.5', 0)

In [ ]:
summary_85.columns

In [ ]:
value_vars = [
    'high_risk_2030', 'very_high_risk_2030', 'extreme_risk_2030',
    'high_risk_2050', 'very_high_risk_2050', 'extreme_risk_2050',
    'high_risk_2080', 'very_high_risk_2080', 'extreme_risk_2080'
]

cu_summary_85_long = summary_85.melt(
    id_vars=['id', 'Gemeinde', 'Kreis', 'Land'],  # keep these
    value_vars=value_vars,
    var_name='risk_year',
    value_name='cu_diff_2020'
)

# Split the combined column into risk_level and year
cu_summary_85_long[['risk_level', 'year']] = cu_summary_85_long['risk_year'].str.extract(r'(.*)_(\d{4})')
cu_summary_85_long['year'] = cu_summary_85_long['year'].astype(int)


### Please store the pre-processed files as pickle files, since you’ll need to load them directly later—especially if the session needs to be restarted due to running out of RAM.

In [ ]:
cu_summary_85_long.to_pickle("cu_summary_85_long.pkl")

In [ ]:
with open('cu_summary_85_long.pkl', 'rb') as f:
    cu_summary_85_long = pickle.load(f)

## Fluvial flooding undefended

### Data Preparation

In [ ]:
# List of years to process
years = [2020, 2030, 2050, 2080]

# Columns you want to select (CU columns and Land)
fu_columns = ['id', 'Gemeinde', 'Kreis', 'Land'] + [col for col in all_columns if col.startswith('FU')]

# Loop through each year and modify the corresponding DataFrame
for year in years:
    if year == 2020:
        df_name = f'Gemeinde_{year}_100_raw'  # 2020 uses '_100_raw'
    else:
        df_name = f'Gemeinde_{year}_100_85_raw'  # 2030, 2050, and 2080 use '_100_85_raw'

    # Check if the DataFrame exists before proceeding
    if df_name in globals():
        # Get the raw DataFrame and select the relevant columns for max flooding data
        df_raw = globals()[df_name]
        globals()[f'Gemeinde_{year}_100_85_FU'] = df_raw[fu_columns]
    else:
        print(f'{df_name} does not exist.')


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    globals()[f'Gemeinde_{y}_100_85_FU_long'] = globals()[f'Gemeinde_{y}_100_85_FU'].melt(
        id_vars=['id', 'Gemeinde', 'Kreis', 'Land'], var_name='month_year_depth', value_name='FU_area'
    )


In [ ]:
Gemeinde_2020_100_85_FU_long.head()

In [ ]:
for df in [Gemeinde_2020_100_85_FU_long, Gemeinde_2030_100_85_FU_long, Gemeinde_2050_100_85_FU_long, Gemeinde_2080_100_85_FU_long]:

    # Replace 'CU' with 'MAX' if needed (optional step if not already 'MAX')
    # df['month_year_depth'] = df['month_year_depth'].str.replace('CU', 'MAX')

    # Split the 'month_year_depth' column into parts
    df[['prefix', 'month_year', 'depth_cat']] = df['month_year_depth'].str.extract(r'^(FU)-([A-Za-z]{3}-\d{2})-(.+)$')

    # Drop 'prefix' if not needed
    df.drop(columns='prefix', inplace=True)


In [ ]:
Gemeinde_2080_100_85_FU_long.head()

In [ ]:
# Define your DataFrames with their corresponding years
year_df_map = {
    2020: Gemeinde_2020_100_85_FU_long,
    2030: Gemeinde_2030_100_85_FU_long,
    2050: Gemeinde_2050_100_85_FU_long,
    2080: Gemeinde_2080_100_85_FU_long
}

# Loop through each DataFrame
for year, df in year_df_map.items():

    # Filter to only Jan-25
    jan25_df = df[df['month_year'] == 'Jan-25'].reset_index(drop=True)

    # Set the new global variable
    globals()[f'Gemeinde_{year}_100_85_FU_Jan25'] = jan25_df


In [ ]:
# Second part: group and create *_settle DataFrames
for year in [2020, 2030, 2050, 2080]:
    jan25_var_name = f'Gemeinde_{year}_100_85_FU_Jan25'

    if jan25_var_name in globals():
        jan25_df = globals()[jan25_var_name]

        # Group and sum by 'id' and 'month_year'
        settle_df = jan25_df.groupby(['id', 'month_year'], as_index=False)['FU_area'].sum()
        settle_df.columns = ['id', 'month_year', 'settle_area']
        settle_df['settle_area'] = settle_df['settle_area'].astype(float)

        # Save to a new global variable
        globals()[f'{jan25_var_name}_settle'] = settle_df

In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    long_df = globals()[f'Gemeinde_{y}_100_85_FU_Jan25']
    settle_df = globals()[f'Gemeinde_{y}_100_85_FU_Jan25_settle']

    merged_df = pd.merge(long_df, settle_df, on=['id', 'month_year'], how='left')

    globals()[f'Gemeinde_{y}_100_85_FU_Jan25'] = merged_df


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    df = globals()[f'Gemeinde_{y}_100_85_FU_Jan25']
    globals()[f'Gemeinde_{y}_100_85_FU_Jan25'] = df
    df['settle_area_non_zero'] = df.settle_area.replace(0, 1e-6)
    globals()[f'Gemeinde_{y}_100_85_FU_Jan25']['settle_area_non_zero'] = df.settle_area_non_zero
    globals()[f'Gemeinde_{y}_100_85_FU_Jan25']['depth_cat_area_pct'] = (df.FU_area / df.settle_area_non_zero) * 100

In [ ]:
cols = ['id', 'Gemeinde', 'Kreis', 'Land']
years = [2030, 2050, 2080]

base_df = Gemeinde_2020_100_85_FU_Jan25

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_85_FU_Jan25']

    # Direct comparison of the column values (row by row, in order)
    is_same = base_df[cols].equals(current_df[cols])

    if is_same:
        print(f"✅ Columns match exactly for year {year}")
    else:
        print(f"❌ Columns differ for year {year}")


In [ ]:
years = [2030, 2050, 2080]

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_85_FU_Jan25']
    base_df = Gemeinde_2020_100_85_FU_Jan25
    base_df[f'diff_{year}'] = current_df.FU_area - base_df.FU_area

In [ ]:
summary = Gemeinde_2020_100_85_FU_Jan25[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates().copy()
years = [2030, 2050, 2080]

for year in years:
    diff_col = f'diff_{year}'

    pivot = (
        Gemeinde_2020_100_85_FU_Jan25
        .groupby(['id', 'depth_cat'])[diff_col]
        .sum()
        .unstack(fill_value=0)
        .add_prefix(f'sum_diff_{year}_')  # e.g., sum_diff_2030_lt0.5
        .reset_index()
    )

    summary = summary.merge(pivot, on='id', how='left')


In [ ]:
# Convert all newly added columns to float
new_cols = [col for col in summary.columns if col.startswith('sum_diff_')]
summary[new_cols] = summary[new_cols].astype(float)

In [ ]:
summary_85 = summary.copy()

In [ ]:
years = [2030, 2050, 2080]

for year in years:
    summary_85[f'high_risk_{year}'] = (
        summary_85.get(f'sum_diff_{year}_lt0.5', 0) +
        summary_85.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_85.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_85[f'very_high_risk_{year}'] = (
        summary_85.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_85.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_85[f'extreme_risk_{year}'] = summary_85.get(f'sum_diff_{year}_gt1.5', 0)

In [ ]:
summary_85.columns

In [ ]:
value_vars = [
    'high_risk_2030', 'very_high_risk_2030', 'extreme_risk_2030',
    'high_risk_2050', 'very_high_risk_2050', 'extreme_risk_2050',
    'high_risk_2080', 'very_high_risk_2080', 'extreme_risk_2080'
]

fu_summary_85_long = summary_85.melt(
    id_vars=['id', 'Gemeinde', 'Kreis', 'Land'],  # keep these
    value_vars=value_vars,
    var_name='risk_year',
    value_name='fu_diff_2020'
)

# Split the combined column into risk_level and year
fu_summary_85_long[['risk_level', 'year']] = fu_summary_85_long['risk_year'].str.extract(r'(.*)_(\d{4})')
fu_summary_85_long['year'] = fu_summary_85_long['year'].astype(int)


### Please store the pre-processed files as pickle files, since you’ll need to load them directly later—especially if the session needs to be restarted due to running out of RAM.

In [ ]:
fu_summary_85_long.to_pickle("fu_summary_85_long.pkl")

In [ ]:
with open('fu_summary_85_long.pkl', 'rb') as f:
    fu_summary_85_long = pickle.load(f)

## Pluvial flooding defended

### Data Preparation

In [ ]:
# List of years to process
years = [2020, 2030, 2050, 2080]

# Columns you want to select (CU columns and Land)
pd_columns = ['id', 'Gemeinde', 'Kreis', 'Land'] + [col for col in all_columns if col.startswith('PD')]

# Loop through each year and modify the corresponding DataFrame
for year in years:
    if year == 2020:
        df_name = f'Gemeinde_{year}_100_raw'  # 2020 uses '_100_raw'
    else:
        df_name = f'Gemeinde_{year}_100_85_raw'  # 2030, 2050, and 2080 use '_100_85_raw'

    # Check if the DataFrame exists before proceeding
    if df_name in globals():
        # Get the raw DataFrame and select the relevant columns for max flooding data
        df_raw = globals()[df_name]
        globals()[f'Gemeinde_{year}_100_85_PD'] = df_raw[pd_columns]
    else:
        print(f'{df_name} does not exist.')


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    globals()[f'Gemeinde_{y}_100_85_PD_long'] = globals()[f'Gemeinde_{y}_100_85_PD'].melt(
        id_vars=['id', 'Gemeinde', 'Kreis', 'Land'], var_name='month_year_depth', value_name='PD_area'
    )


In [ ]:
Gemeinde_2020_100_85_PD_long.head()

In [ ]:
for df in [Gemeinde_2020_100_85_PD_long, Gemeinde_2030_100_85_PD_long, Gemeinde_2050_100_85_PD_long, Gemeinde_2080_100_85_PD_long]:

    # Replace 'CU' with 'MAX' if needed (optional step if not already 'MAX')
    # df['month_year_depth'] = df['month_year_depth'].str.replace('CU', 'MAX')

    # Split the 'month_year_depth' column into parts
    df[['prefix', 'month_year', 'depth_cat']] = df['month_year_depth'].str.extract(r'^(PD)-([A-Za-z]{3}-\d{2})-(.+)$')

    # Drop 'prefix' if not needed
    df.drop(columns='prefix', inplace=True)


In [ ]:
Gemeinde_2080_100_85_PD_long.head()

In [ ]:
# Define your DataFrames with their corresponding years
year_df_map = {
    2020: Gemeinde_2020_100_85_PD_long,
    2030: Gemeinde_2030_100_85_PD_long,
    2050: Gemeinde_2050_100_85_PD_long,
    2080: Gemeinde_2080_100_85_PD_long
}

# Loop through each DataFrame
for year, df in year_df_map.items():

    # Filter to only Jan-25
    jan25_df = df[df['month_year'] == 'Jan-25'].reset_index(drop=True)

    # Set the new global variable
    globals()[f'Gemeinde_{year}_100_85_PD_Jan25'] = jan25_df


In [ ]:
# Second part: group and create *_settle DataFrames
for year in [2020, 2030, 2050, 2080]:
    jan25_var_name = f'Gemeinde_{year}_100_85_PD_Jan25'

    if jan25_var_name in globals():
        jan25_df = globals()[jan25_var_name]

        # Group and sum by 'id' and 'month_year'
        settle_df = jan25_df.groupby(['id', 'month_year'], as_index=False)['PD_area'].sum()
        settle_df.columns = ['id', 'month_year', 'settle_area']
        settle_df['settle_area'] = settle_df['settle_area'].astype(float)

        # Save to a new global variable
        globals()[f'{jan25_var_name}_settle'] = settle_df

In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    long_df = globals()[f'Gemeinde_{y}_100_85_PD_Jan25']
    settle_df = globals()[f'Gemeinde_{y}_100_85_PD_Jan25_settle']

    merged_df = pd.merge(long_df, settle_df, on=['id', 'month_year'], how='left')

    globals()[f'Gemeinde_{y}_100_85_PD_Jan25'] = merged_df


In [ ]:
years = [2020, 2030, 2050, 2080]

for y in years:
    df = globals()[f'Gemeinde_{y}_100_85_PD_Jan25']
    globals()[f'Gemeinde_{y}_100_85_PD_Jan25'] = df
    df['settle_area_non_zero'] = df.settle_area.replace(0, 1e-6)
    globals()[f'Gemeinde_{y}_100_85_PD_Jan25']['settle_area_non_zero'] = df.settle_area_non_zero
    globals()[f'Gemeinde_{y}_100_85_PD_Jan25']['depth_cat_area_pct'] = (df.PD_area / df.settle_area_non_zero) * 100

In [ ]:
cols = ['id', 'Gemeinde', 'Kreis', 'Land']
years = [2030, 2050, 2080]

base_df = Gemeinde_2020_100_85_PD_Jan25

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_85_PD_Jan25']

    # Direct comparison of the column values (row by row, in order)
    is_same = base_df[cols].equals(current_df[cols])

    if is_same:
        print(f"✅ Columns match exactly for year {year}")
    else:
        print(f"❌ Columns differ for year {year}")


In [ ]:
years = [2030, 2050, 2080]

for year in years:
    current_df = globals()[f'Gemeinde_{year}_100_85_PD_Jan25']
    base_df = Gemeinde_2020_100_85_PD_Jan25
    base_df[f'diff_{year}'] = current_df.PD_area - base_df.PD_area

In [ ]:
summary = Gemeinde_2020_100_85_PD_Jan25[['id', 'Gemeinde', 'Kreis', 'Land']].drop_duplicates().copy()
years = [2030, 2050, 2080]

for year in years:
    diff_col = f'diff_{year}'

    pivot = (
        Gemeinde_2020_100_85_PD_Jan25
        .groupby(['id', 'depth_cat'])[diff_col]
        .sum()
        .unstack(fill_value=0)
        .add_prefix(f'sum_diff_{year}_')  # e.g., sum_diff_2030_lt0.5
        .reset_index()
    )

    summary = summary.merge(pivot, on='id', how='left')


In [ ]:
# Convert all newly added columns to float
new_cols = [col for col in summary.columns if col.startswith('sum_diff_')]
summary[new_cols] = summary[new_cols].astype(float)

In [ ]:
summary_85 = summary.copy()

In [ ]:
years = [2030, 2050, 2080]

for year in years:
    summary_85[f'high_risk_{year}'] = (
        summary_85.get(f'sum_diff_{year}_lt0.5', 0) +
        summary_85.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_85.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_85[f'very_high_risk_{year}'] = (
        summary_85.get(f'sum_diff_{year}_lt1.5', 0) +
        summary_85.get(f'sum_diff_{year}_gt1.5', 0)
    )

    summary_85[f'extreme_risk_{year}'] = summary_85.get(f'sum_diff_{year}_gt1.5', 0)

In [ ]:
summary_85.columns

In [ ]:
value_vars = [
    'high_risk_2030', 'very_high_risk_2030', 'extreme_risk_2030',
    'high_risk_2050', 'very_high_risk_2050', 'extreme_risk_2050',
    'high_risk_2080', 'very_high_risk_2080', 'extreme_risk_2080'
]

pd_summary_85_long = summary_85.melt(
    id_vars=['id', 'Gemeinde', 'Kreis', 'Land'],  # keep these
    value_vars=value_vars,
    var_name='risk_year',
    value_name='pd_diff_2020'
)

# Split the combined column into risk_level and year
pd_summary_85_long[['risk_level', 'year']] = pd_summary_85_long['risk_year'].str.extract(r'(.*)_(\d{4})')
pd_summary_85_long['year'] = pd_summary_85_long['year'].astype(int)


### Please store the pre-processed files as pickle files, since you’ll need to load them directly later—especially if the session needs to be restarted due to running out of RAM.

In [ ]:
pd_summary_85_long.to_pickle("pd_summary_85_long.pkl")

In [ ]:
with open("pd_summary_85_long.pkl", "rb") as f:
    pd_summary_85_long = pickle.load(f)

# Combined Gemeinden Plot

In [ ]:
# Load needed the processed variables
with open("max_summary_26_long.pkl", "rb") as f:
    summary_26_long = pickle.load(f)

with open("max_summary_85_long.pkl", "rb") as f:
    summary_85_long = pickle.load(f)


In [ ]:
sns.set_style("whitegrid")
# Only the first and last scenarios
scenarios = ['SSP1-RCP2.6', 'SSP5-RCP8.5']
scenario_files = ['summary_26_long', 'summary_85_long']

# Setup
save_path = "/content/drive/MyDrive/Germany_Flood_Study/Gemeinde_climate_scenario_plots/final_adjustments"
os.makedirs(save_path, exist_ok=True)
file_name = "separate_risk_level_future_exposure_diff_Jan25_stacked_extreme_scenarios_strip.png"
full_path = os.path.join(save_path, file_name)

# Color and labels
red_shades = ['#ff9999', '#ff1a1a', '#800000']
risk_order = ['high_risk', 'very_high_risk', 'extreme_risk']
risk_label_map = {
    'high_risk': 'Flood depth > 0.15 m',
    'very_high_risk': 'Flood depth > 0.5 m',
    'extreme_risk': 'Flood depth > 1.5 m'
}
color_map = dict(zip(risk_order, red_shades))

# 2 rows x 3 cols
fig, axs = plt.subplots(2, 3, figsize=(18, 12), sharey=True)
sns.set(style="whitegrid")

for row_idx, (scenario, data_name) in enumerate(zip(scenarios, scenario_files)):
    data = globals()[data_name]
    for col_idx, risk in enumerate(risk_order):
        ax = axs[row_idx, col_idx]
        subset = data[(data['risk_level'] == risk) & (data['Gemeinde'] != 'Germany')]

        sns.stripplot(
            data=subset,
            x='year',
            y='max_diff_2020',
            color=color_map[risk],
            s=2,
            alpha=0.1,
            ax=ax,
            zorder=-1
        )

        # Percentiles
        grouped = subset.groupby('year')['max_diff_2020']
        for rp, group in grouped:
            percentiles = {
                '50': group.median(),
                '75': group.quantile(0.75),
                '90': group.quantile(0.90)
            }
            xpos = list(subset['year'].unique()).index(rp)
            ax.hlines(percentiles['50'], xpos - 0.15, xpos + 0.15, color='black', linewidth=1.5, linestyles='solid')
            ax.hlines(percentiles['75'], xpos - 0.2, xpos + 0.2, color='black', linewidth=1.2, linestyles='dashed')
            ax.hlines(percentiles['90'], xpos - 0.3, xpos + 0.3, color='black', linewidth=1, linestyles='dotted')

        # Titles & Labels
        if row_idx == 0:
            ax.set_title(risk_label_map[risk], fontsize=16)
        if col_idx == 0:
            ax.set_ylabel(f"{scenario}\n\nFlood exposure change (km$^2$)", fontsize=16)
        else:
            ax.set_ylabel("")
        ax.set_xlabel("")
        ax.set_ylim(-0.01, 0.05)
        ax.tick_params(axis='x', rotation=45, labelsize=14)
        ax.tick_params(axis='y', labelsize=14)

# Custom legend at bottom middle
legend_lines = [
    Line2D([0], [0], color='black', linewidth=1.5, linestyle='solid', label='50th percentile'),
    Line2D([0], [0], color='black', linewidth=1.2, linestyle='dashed', label='75th percentile'),
    Line2D([0], [0], color='black', linewidth=1, linestyle='dotted', label='90th percentile')
]

fig.legend(
    handles=legend_lines,
    # title='Percentile',
    loc='lower center',
    bbox_to_anchor=(0.5, -0.05),
    ncol=3,
    fontsize=16,
    title_fontsize=18
)

# fig.supxlabel("Projected Years", fontsize=16)
plt.tight_layout()
plt.subplots_adjust(bottom=0.12)  # make space for the legend

# plt.savefig(full_path, dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Only take the first and last scenario
scenarios_subset = ['SSP1-RCP2.6', 'SSP5-RCP8.5']
scenario_files_subset = ['summary_26_long', 'summary_85_long']

# Setup
save_path = "/content/drive/MyDrive/Germany_Flood_Study/Gemeinde_climate_scenario_plots/final_adjustments"
os.makedirs(save_path, exist_ok=True)
file_name = "separate_risk_level_future_exposure_pct_diff_Jan25_stacked_extreme_scenarios_strip.png"
full_path = os.path.join(save_path, file_name)

# Color and labels
red_shades = ['#ff9999', '#ff1a1a', '#800000']
risk_order = ['high_risk', 'very_high_risk', 'extreme_risk']
risk_label_map = {
    'high_risk': 'Flood depth > 0.15 m',
    'very_high_risk': 'Flood depth > 0.5 m',
    'extreme_risk': 'Flood depth > 1.5 m'
}
color_map = dict(zip(risk_order, red_shades))

fig, axs = plt.subplots(2, 3, figsize=(18, 12), sharey=True)  # 2 rows now
sns.set(style="whitegrid")

for row_idx, (scenario, data_name) in enumerate(zip(scenarios_subset, scenario_files_subset)):
    data = globals()[data_name]
    for col_idx, risk in enumerate(risk_order):
        ax = axs[row_idx, col_idx]
        subset = data[
            (data['risk_level'] == risk) &
            (data['Gemeinde'] != 'Germany')
        ]

        sns.stripplot(
            data=subset,
            x='year',
            y='max_pct_diff_2020',
            color=color_map[risk],
            s=2,
            alpha=0.1,
            ax=ax,
            zorder=-1
        )

        # Draw percentiles
        grouped = subset.groupby('year')['max_pct_diff_2020']
        for rp, group in grouped:
            percentiles = {
                '50': group.median(),
                '75': group.quantile(0.75),
                '90': group.quantile(0.90)
            }
            xpos = list(subset['year'].unique()).index(rp)
            ax.hlines(percentiles['50'], xpos - 0.15, xpos + 0.15, color='black', linewidth=1.5, linestyles='solid')
            ax.hlines(percentiles['75'], xpos - 0.2, xpos + 0.2, color='black', linewidth=1.2, linestyles='dashed')
            ax.hlines(percentiles['90'], xpos - 0.3, xpos + 0.3, color='black', linewidth=1, linestyles='dotted')

        # Titles & Labels
        if row_idx == 0:
            ax.set_title(risk_label_map[risk], fontsize=16)
        if col_idx == 0:
            ax.set_ylabel(f"{scenario}\n\nFlood exposure change (%)", fontsize=14)
        else:
            ax.set_ylabel("")
        ax.set_xlabel("")
        ax.set_ylim(-25, 175)
        ax.tick_params(axis='x', rotation=45)

# Legend at bottom middle
legend_lines = [
    Line2D([0], [0], color='black', linewidth=1.5, linestyle='solid', label='50th percentile'),
    Line2D([0], [0], color='black', linewidth=1.2, linestyle='dashed', label='75th percentile'),
    Line2D([0], [0], color='black', linewidth=1, linestyle='dotted', label='90th percentile')
]

fig.legend(
    handles=legend_lines,
    # title='Percentile',
    loc='lower center',
    bbox_to_anchor=(0.5, -0.05),
    ncol=3,
    fontsize=16,
    title_fontsize=18
)

# fig.suptitle("Max Flood Exposure %Change by Depth Across Climate Scenarios (Relative to 2020, Jan-25)", fontsize=20, y=1.02)
# fig.supxlabel("Projected Years", fontsize=16)

plt.tight_layout()
# plt.savefig(full_path, dpi=300, bbox_inches='tight')
plt.show()


# National Level Plot

In [ ]:
# Load all the processed variables
with open("max_summary_26_long.pkl", "rb") as f:
    summary_26_long = pickle.load(f)
with open("cu_summary_26_long.pkl", "rb") as f:
    cu_summary_26_long = pickle.load(f)
with open("fu_summary_26_long.pkl", "rb") as f:
    fu_summary_26_long = pickle.load(f)
with open("pd_summary_26_long.pkl", "rb") as f:
    pd_summary_26_long = pickle.load(f)

with open("max_summary_45_long.pkl", "rb") as f:
    summary_45_long = pickle.load(f)
with open("cu_summary_45_long.pkl", "rb") as f:
    cu_summary_45_long = pickle.load(f)
with open("fu_summary_45_long.pkl", "rb") as f:
    fu_summary_45_long = pickle.load(f)
with open("pd_summary_45_long.pkl", "rb") as f:
    pd_summary_45_long = pickle.load(f)

with open("max_summary_70_long.pkl", "rb") as f:
    summary_70_long = pickle.load(f)
with open("cu_summary_70_long.pkl", "rb") as f:
    cu_summary_70_long = pickle.load(f)
with open("fu_summary_70_long.pkl", "rb") as f:
    fu_summary_70_long = pickle.load(f)
with open("pd_summary_70_long.pkl", "rb") as f:
    pd_summary_70_long = pickle.load(f)

with open("max_summary_85_long.pkl", "rb") as f:
    summary_85_long = pickle.load(f)
with open("cu_summary_85_long.pkl", "rb") as f:
    cu_summary_85_long = pickle.load(f)
with open("fu_summary_85_long.pkl", "rb") as f:
    fu_summary_85_long = pickle.load(f)
with open("pd_summary_85_long.pkl", "rb") as f:
    pd_summary_85_long = pickle.load(f)

In [ ]:
# Setup
save_path = "/content/drive/MyDrive/Germany_Flood_Study/Gemeinde_climate_scenario_plots/final_adjustments"
os.makedirs(save_path, exist_ok=True)
file_name = "separate_risk_level_future_exposure_diff_Jan25_Germany.png"
full_path = os.path.join(save_path, file_name)

# Scenarios and corresponding dataframes
scenarios = ['26', '45', '70', '85']
scenario_titles = {
    '26': 'SSP1-RCP2.6',
    '45': 'SSP2-RCP4.5',
    '70': 'SSP3-RCP7.0',
    '85': 'SSP5-RCP8.5'
}

# Risk levels and colors
risk_order = ['high_risk', 'very_high_risk', 'extreme_risk']
risk_label_map = {
    'high_risk': '> 0.15 m',
    'very_high_risk': '> 0.5 m',
    'extreme_risk': '> 1.5 m'
}
red_shades = ['#ff9999', '#ff1a1a', '#800000']
color_map = dict(zip(risk_order, red_shades))

# Create 1x4 subplots
fig, axs = plt.subplots(1, 4, figsize=(22, 6), sharey=True)

for i, scenario in enumerate(scenarios):
    df = globals()[f'summary_{scenario}_long']
    df_germany = df[df['Gemeinde'] == 'Germany'].copy()

    # Add a 2020 zero point for each risk level
    zero_df = pd.DataFrame({
        'year': [2020] * len(risk_order),
        'max_diff_2020': [0] * len(risk_order),
        'risk_level': risk_order,
        'Gemeinde': ['Germany'] * len(risk_order)
    })

    df_germany = pd.concat([zero_df, df_germany], ignore_index=True)

    sns.lineplot(
        data=df_germany,
        x='year',
        y='max_diff_2020',
        hue='risk_level',
        hue_order=risk_order,
        palette=color_map,
        marker='o',
        ax=axs[i]
    )

    axs[i].set_title(scenario_titles[scenario], fontsize=16)
    axs[i].set_xlabel('Year', fontsize=14)
    axs[i].set_xticks([2020, 2030, 2050, 2080])
    axs[i].set_ylim(-10, 240)
    axs[i].grid(True)

    if i == 0:
        axs[i].set_ylabel('Flood exposure increase (km$^2$)', fontsize=16, rotation=90)
    else:
        axs[i].set_ylabel('')

    axs[i].get_legend().remove()

# Create a single legend from the first axis
handles, labels = axs[0].get_legend_handles_labels()
new_labels = [risk_label_map.get(label, label) for label in labels]

# Add legend at bottom center
fig.legend(
    handles,
    new_labels,
    title='Flood depth',
    loc='lower center',
    bbox_to_anchor=(0.5, -0.08),
    ncol=len(risk_order),
    fontsize=16,
    title_fontsize=18
)

plt.tight_layout(rect=[0, 0.05, 1, 0.95])  # leave space for bottom legend

# plt.savefig(full_path, dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Setup
save_path = "/content/drive/MyDrive/Germany_Flood_Study/Gemeinde_climate_scenario_plots/final_adjustments"
os.makedirs(save_path, exist_ok=True)
file_name = "separate_risk_level_future_cu_exposure_diff_Jan25_Germany.png"
full_path = os.path.join(save_path, file_name)

# Scenarios and corresponding dataframes
scenarios = ['26', '45', '70', '85']
scenario_titles = {
    '26': 'SSP1-RCP2.6',
    '45': 'SSP2-RCP4.5',
    '70': 'SSP3-RCP7.0',
    '85': 'SSP5-RCP8.5'
}

# Risk levels and colors
risk_order = ['high_risk', 'very_high_risk', 'extreme_risk']
risk_label_map = {
    'high_risk': '> 0.15 m',
    'very_high_risk': '> 0.5 m',
    'extreme_risk': '> 1.5 m'
}
blue_shades = ['#99ccff', '#1a75ff', '#003366']
color_map = dict(zip(risk_order, blue_shades))

# Create 1x4 subplots
fig, axs = plt.subplots(1, 4, figsize=(22, 6), sharey=True)

for i, scenario in enumerate(scenarios):
    df = globals()[f'cu_summary_{scenario}_long']
    df_germany = df[df['Gemeinde'] == 'Germany'].copy()

    # Add a 2020 zero point for each risk level
    zero_df = pd.DataFrame({
        'year': [2020] * len(risk_order),
        'cu_diff_2020': [0] * len(risk_order),
        'risk_level': risk_order,
        'Gemeinde': ['Germany'] * len(risk_order)
    })

    # Combine with the original data
    df_germany = pd.concat([zero_df, df_germany], ignore_index=True)

    sns.lineplot(
        data=df_germany,
        x='year',
        y='cu_diff_2020',
        hue='risk_level',
        hue_order=risk_order,
        palette=color_map,
        marker='o',
        ax=axs[i]
    )

    axs[i].set_title(scenario_titles[scenario])
    axs[i].set_xlabel('Year', fontsize=14)
    axs[i].set_xticks([2020, 2030, 2050, 2080])
    # axs[i].set_ylim(-10, 240)
    axs[i].grid(True)

    if i == 0:
        axs[i].set_ylabel('Flood exposure increase (km$^2$)', fontsize=16, rotation=90)
    else:
        axs[i].set_ylabel('')

    axs[i].get_legend().remove()

# Create a single legend from the first axis
handles, labels = axs[0].get_legend_handles_labels()
# Map the risk levels to custom labels
new_labels = [risk_label_map.get(label, label) for label in labels]

# Add legend outside the plot
fig.legend(
    handles,
    new_labels,
    title='Flood depth',
    loc='lower center',
    bbox_to_anchor=(0.5, -0.12),
    ncol=len(risk_order),
    fontsize=16,
    title_fontsize=18  # smaller x value = closer to the plot
)

plt.tight_layout(rect=[0, 0, 0.9, 1])

# Save or show the plot
# plt.savefig(full_path, dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Setup
save_path = "/content/drive/MyDrive/Germany_Flood_Study/Gemeinde_climate_scenario_plots/final_adjustments"
os.makedirs(save_path, exist_ok=True)
file_name = "separate_risk_level_future_fu_exposure_diff_Jan25_Germany.png"
full_path = os.path.join(save_path, file_name)

# Scenarios and corresponding dataframes
scenarios = ['26', '45', '70', '85']
scenario_titles = {
    '26': 'SSP1-RCP2.6',
    '45': 'SSP2-RCP4.5',
    '70': 'SSP3-RCP7.0',
    '85': 'SSP5-RCP8.5'
}

# Risk levels and colors
risk_order = ['high_risk', 'very_high_risk', 'extreme_risk']
risk_label_map = {
    'high_risk': '> 0.15 m',
    'very_high_risk': '> 0.5 m',
    'extreme_risk': '> 1.5 m'
}
green_shades = ['#a8e6a3', '#4caf50', '#1b5e20']
color_map = dict(zip(risk_order, green_shades))

# Create 1x4 subplots
fig, axs = plt.subplots(1, 4, figsize=(22, 6), sharey=True)

for i, scenario in enumerate(scenarios):
    df = globals()[f'fu_summary_{scenario}_long']
    df_germany = df[df['Gemeinde'] == 'Germany'].copy()

    # Add a 2020 zero point for each risk level
    zero_df = pd.DataFrame({
        'year': [2020] * len(risk_order),
        'fu_diff_2020': [0] * len(risk_order),
        'risk_level': risk_order,
        'Gemeinde': ['Germany'] * len(risk_order)
    })

    # Combine with the original data
    df_germany = pd.concat([zero_df, df_germany], ignore_index=True)

    sns.lineplot(
        data=df_germany,
        x='year',
        y='fu_diff_2020',
        hue='risk_level',
        hue_order=risk_order,
        palette=color_map,
        marker='o',
        ax=axs[i]
    )

    axs[i].set_title(scenario_titles[scenario])
    axs[i].set_xlabel('Year', fontsize=14)
    axs[i].set_xticks([2020, 2030, 2050, 2080])
    # axs[i].set_ylim(-10, 240)
    axs[i].grid(True)

    if i == 0:
        axs[i].set_ylabel('Flood exposure increase (km$^2$)', fontsize=16, rotation=90)
    else:
        axs[i].set_ylabel('')

    axs[i].get_legend().remove()

# Create a single legend from the first axis
handles, labels = axs[0].get_legend_handles_labels()
# Map the risk levels to custom labels
new_labels = [risk_label_map.get(label, label) for label in labels]

# Add legend outside the plot
fig.legend(
    handles,
    new_labels,
    title='Flood depth',
    loc='lower center',
    bbox_to_anchor=(0.5, -0.12),
    ncol=len(risk_order),
    fontsize=16,
    title_fontsize=18
)


plt.tight_layout(rect=[0, 0, 0.9, 1])

# Save or show the plot
# plt.savefig(full_path, dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Setup
save_path = "/content/drive/MyDrive/Germany_Flood_Study/Gemeinde_climate_scenario_plots/final_adjustments"
os.makedirs(save_path, exist_ok=True)
file_name = "separate_risk_level_future_pd_exposure_diff_Jan25_Germany.png"
full_path = os.path.join(save_path, file_name)

# Scenarios and corresponding dataframes
scenarios = ['26', '45', '70', '85']
scenario_titles = {
    '26': 'SSP1-RCP2.6',
    '45': 'SSP2-RCP4.5',
    '70': 'SSP3-RCP7.0',
    '85': 'SSP5-RCP8.5'
}

# Risk levels and colors
risk_order = ['high_risk', 'very_high_risk', 'extreme_risk']
risk_label_map = {
    'high_risk': '> 0.15 m',
    'very_high_risk': '> 0.5 m',
    'extreme_risk': '> 1.5 m'
}
purple_shades = ['#d1b3ff', '#9c27b0', '#4a0072']
color_map = dict(zip(risk_order, purple_shades))

# Create 1x4 subplots
fig, axs = plt.subplots(1, 4, figsize=(22, 6), sharey=True)

for i, scenario in enumerate(scenarios):
    df = globals()[f'pd_summary_{scenario}_long']
    df_germany = df[df['Gemeinde'] == 'Germany'].copy()

    # Add a 2020 zero point for each risk level
    zero_df = pd.DataFrame({
        'year': [2020] * len(risk_order),
        'pd_diff_2020': [0] * len(risk_order),
        'risk_level': risk_order,
        'Gemeinde': ['Germany'] * len(risk_order)
    })

    # Combine with the original data
    df_germany = pd.concat([zero_df, df_germany], ignore_index=True)

    sns.lineplot(
        data=df_germany,
        x='year',
        y='pd_diff_2020',
        hue='risk_level',
        hue_order=risk_order,
        palette=color_map,
        marker='o',
        ax=axs[i]
    )

    axs[i].set_title(scenario_titles[scenario])
    axs[i].set_xlabel('Year', fontsize=14)
    axs[i].set_xticks([2020, 2030, 2050, 2080])
    # axs[i].set_ylim(-10, 240)
    axs[i].grid(True)

    if i == 0:
        axs[i].set_ylabel('Flood exposure increase (km$^2$)', fontsize=16, rotation=90)
    else:
        axs[i].set_ylabel('')

    axs[i].get_legend().remove()

# Create a single legend from the first axis
handles, labels = axs[0].get_legend_handles_labels()
# Map the risk levels to custom labels
new_labels = [risk_label_map.get(label, label) for label in labels]

# Add legend outside the plot
fig.legend(
    handles,
    new_labels,
    title='Flood depth',
    loc='lower center',
    bbox_to_anchor=(0.5, -0.12),
    ncol=len(risk_order),
    fontsize=16,
    title_fontsize=18
)

plt.tight_layout(rect=[0, 0, 0.9, 1])
# Save or show the plot
# plt.savefig(full_path, dpi=300, bbox_inches='tight')
plt.show()
